# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체게 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [39]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

In [6]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [200]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [536]:
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
pub_fold = 'D:/data/processing/big3/raw/public'
folds = {'pub_fold':pub_fold, 'raw_fold':raw_fold, 'an_fold':an_fold, 'df_fold':df_fold, 
         'raw2_fold':raw2_fold, 'raw4_fold':raw4_fold, 'raw5_fold':raw5_fold, 
         'an2_fold:':an2_fold, 'an4_fold:':an4_fold, 'an5_fold:':an5_fold, 
         'df2_fold':df2_fold, 'df4_fold':df4_fold, 'df5_fold':df5_fold}
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'an_fold': 'D:/data/processing/big3/analysis',
 'df_fold': 'D:/data/processing/big3/df',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# EDA
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

## 과제2 > 건설기계 인증자료(12개)

In [81]:
def construct_machine_certification_data_load(name, sheet_name=0):
    try:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name), sheet_name=sheet_name)
        return cert_dt
    except:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name))
        return cert_dt

### 정보 모음

In [93]:
cols = construct_machine_certification_data_load('KENCIS_건설기계인증정보_테이블명세서', '테이블정의서')
cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   테이블명    12 non-null     object
 1   테이블설명   12 non-null     object
 2   컬럼명     649 non-null    object
 3   컬럼설명    644 non-null    object
 4   데이터형식   649 non-null    object
 5   길이      499 non-null    object
 6   Null    649 non-null    object
dtypes: object(7)
memory usage: 35.6+ KB


In [94]:
cols[['테이블명', '테이블설명']] = cols[['테이블명', '테이블설명']].ffill()

In [199]:
name_list = [
    'UNI_FC_TCMASTER', 
    'UNI_FC_TCERTIMODI', 
    'UNI_FC_ENG_TEFSPEC', 
    'UNI_FC_TEEMISSN', 
    'UNI_FC_TEEMISSN_ENG_RES', 
    'UNI_FC_TESTINFO', 
    'UNI_FC_ENG_LIST', 
    'UNI_FC_ENDUENG_EMI', 
    'UNI_FC_ENDUENGTEST', 
    'UNI_FC_ENDUENGTEST_RES', 
    'UNI_FC_KIENGTEST', 
    'UNI_FC_KIENGTEST_RES', 
    ]
len(name_list)

12

In [110]:
cols_dict = {}
for t in name_list:
    cols_dict[t] = {a:b for a, b in cols.loc[cols['테이블명'] == t, ['컬럼명', '컬럼설명']].values}

In [111]:
cols_dict

{'UNI_FC_ENDUENG_EMI': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'CO_GRAD': 'CO기울기',
  'CO_YITCT': 'CO Y절편',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'NMHC_GRAD': 'NMHC기울기',
  'NMHC_YITCT': 'NMHC Y절편',
  'NOX_GRAD': 'NOX기울기',
  'NOX_YITCT': 'NOX Y절편',
  'PM_GRAD': 'PM기울기',
  'PM_YITCT': 'PM Y절편',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MOD': '배출가스시험모드',
  'TEST_RST_SNO': '시험결과일련번호',
  'PN_GRAD': 'PN 기울기',
  'PN_YITCT': 'PN Y절편'},
 'UNI_FC_ENDUENGTEST': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'ENG_ID_NO': '대표원동기명(원동기코드)',
  'ENG_NO': '원동기 번호',
  'ENG_TYPE': '엔진형식',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_DATE': '시험일',
  'TEST_ENG_SNO': '시험원동기일련번호',
  'TEST_NO': '시험번호'},
 'UNI_FC_ENDUENGTEST_RES': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '인증 일련번호',
  'P_ENG_YN': '내구대표차원동기여부',
  'SEL_ADM_NO': '내구 동일원동기  관리번호',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MO

In [192]:
name = 'UNI_FC_KIENGTEST_RES'
ct01 = construct_machine_certification_data_load(name)
ct01.info()

c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [187]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'CERTI_MODI_SNO': '변경인증일련번호',
 'EF_SNO': '동일차종일련번호',
 'ENG_DISPL': '배기량',
 'ENG_ID_NO': '대표원동기명(원동기코드)',
 'ENG_SNO': '원동기장착일련번호',
 'ENG_TYPE': '엔진형식',
 'ETC': '기타 정보',
 'MAX_PWR_KW': '최대출력kW',
 'MAX_PWR_RPM': '최대출력 회전수',
 'MOD_DATETIME': '수정일시',
 'MOD_ID': '수정아이디',
 'MODI_REF_EF_SNO': '변경부모동일원동기번호',
 'REG_DATETIME': '등록일시',
 'REG_ID': '등록아이디',
 'VEH_IN_COM': '수입사',
 'VEH_MAKER': '제작사',
 'VEH_NM': '차명',
 'VEH_PRE_NM': '건설(농)기계 수입사 대표자 성명',
 'VEH_TYPE': '자동차형식',
 'VEH_TYPE_YN': '인증서 표출 여부'}

In [189]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'CERTI_MODI_SNO': '변경인증일련번호',
 'EF_SNO': '동일차종일련번호',
 'ENG_DISPL': '배기량',
 'ENG_ID_NO': '대표원동기명(원동기코드)',
 'ENG_SNO': '원동기장착일련번호',
 'ENG_TYPE': '엔진형식',
 'ETC': '기타 정보',
 'MAX_PWR_KW': '최대출력kW',
 'MAX_PWR_RPM': '최대출력 회전수',
 'MOD_DATETIME': '수정일시',
 'MOD_ID': '수정아이디',
 'MODI_REF_EF_SNO': '변경부모동일원동기번호',
 'REG_DATETIME': '등록일시',
 'REG_ID': '등록아이디',
 'VEH_IN_COM': '수입사',
 'VEH_MAKER': '제작사',
 'VEH_NM': '차명',
 'VEH_PRE_NM': '건설(농)기계 수입사 대표자 성명',
 'VEH_TYPE': '자동차형식',
 'VEH_TYPE_YN': '인증서 표출 여부'}

In [190]:
cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
cols_df

,컬럼명,컬럼설명
0,ADM_NO,관리번호
1,CERTI_MODI_SNO,변경인증일련번호
2,EF_SNO,동일차종일련번호
3,ENG_DISPL,배기량
4,ENG_ID_NO,대표원동기명(원동기코드)
5,ENG_SNO,원동기장착일련번호
6,ENG_TYPE,엔진형식
7,ETC,기타 정보
8,MAX_PWR_KW,최대출력kW
9,MAX_PWR_RPM,최대출력 회전수


In [191]:
temp = ct01.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = ct01.shape[0]
temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
ttemp = temp.merge(cols_df, on='컬럼명', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,ENG_SNO,원동기장착일련번호,0,9521
1,2,ADM_NO,관리번호,0,9521
2,3,CERTI_MODI_SNO,변경인증일련번호,0,9521
3,4,EF_SNO,동일차종일련번호,0,9521
4,5,ENG_TYPE,엔진형식,9516,9521
5,6,ENG_ID_NO,대표원동기명(원동기코드),9516,9521
6,7,MAX_PWR_KW,최대출력kW,9516,9521
7,8,MAX_PWR_RPM,최대출력 회전수,9516,9521
8,9,ENG_DISPL,배기량,9516,9521
9,10,VEH_IN_COM,수입사,240,9521


In [195]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

#### 컬럼, 데이터 정보들 출력

In [201]:
# 41.2s
with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증자료_데이터정보({today_date}).xlsx')) as writer:
    for i, name in enumerate(name_list):
        cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
        ct = construct_machine_certification_data_load(name)
        temp = ct.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
        temp['총 행수'] = ct.shape[0]
        temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
        ttemp = temp.merge(cols_df, on='컬럼명', how='left')
        ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
        ttemp.to_excel(writer, sheet_name=f'{i+1}. {name}', index=False)

In [202]:
name_list

['UNI_FC_TCMASTER',
 'UNI_FC_TCERTIMODI',
 'UNI_FC_ENG_TEFSPEC',
 'UNI_FC_TEEMISSN',
 'UNI_FC_TEEMISSN_ENG_RES',
 'UNI_FC_TESTINFO',
 'UNI_FC_ENG_LIST',
 'UNI_FC_ENDUENG_EMI',
 'UNI_FC_ENDUENGTEST',
 'UNI_FC_ENDUENGTEST_RES',
 'UNI_FC_KIENGTEST',
 'UNI_FC_KIENGTEST_RES']

### 1\. UNI_FC_TCMASTER

In [203]:
name = name_list[0]
name

'UNI_FC_TCMASTER'

In [220]:
ct01 = construct_machine_certification_data_load(name)
ct01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


#### 유니크 값 탐색

In [ ]:
# CERTI_DIV(인증구분) : 1, 2
# EMIS_CERTI_NO(배출가스인증번호) : 00EN*AA*00 형식(숫자, 영문, 특수문자 혼합 10자리)

In [246]:
temp = ct01.iloc[:, :15]
temp.head()

,ADM_NO,FACT_SNO,CERTI_DIV,REG_DATETIME,RECT_DATE,EMIS_CERTI_NO,NOISE_CERTI_NO,EMIS_OMIT_NO,NOISE_OMIT_NO,EMIS_CERTI_DATE,NOISE_CERTI_DATE,APPLI_ID,EF_CD,OFFICER_ID,PROC_STATE
0,54831,NaN,1,19/01/24 08:47:12.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5553DDRND1,NaN,NaN,NaN
1,54861,NaN,1,19/01/31 16:26:07.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5553DDRND1,NaN,NaN,NaN
2,55026,NaN,2,19/04/19 14:16:55.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5553volvo1,NaN,NaN,NaN
3,55027,NaN,2,19/04/19 14:22:44.000000000,19/05/27 10:52:20.000000000,19EN*VO*01,NaN,NaN,NaN,19/05/28 00:00:00.000000000,NaN,5553volvo1,NaN,NaN,NaN
4,55028,NaN,2,19/04/19 14:46:46.000000000,19/05/27 10:56:43.000000000,19EN*VO*02,NaN,NaN,NaN,19/05/28 00:00:00.000000000,NaN,5553volvo1,NaN,NaN,NaN


In [247]:
for one in temp.columns:
    print(one, cols_dict[name][one])

ADM_NO 관리번호
FACT_SNO 사업장
CERTI_DIV 인증구분
REG_DATETIME 등록일시
RECT_DATE 접수일
EMIS_CERTI_NO 배출인증번호
NOISE_CERTI_NO 소음인증번호
EMIS_OMIT_NO 배출가스인증생략번호
NOISE_OMIT_NO 소음 인증생략번호
EMIS_CERTI_DATE 배출가스인증일 
NOISE_CERTI_DATE 소음인증일
APPLI_ID 신청아이디
EF_CD 동일차종기호
OFFICER_ID 담당자 ID
PROC_STATE 인증처리상태


In [248]:
temp.isnull().sum()

ADM_NO                0
FACT_SNO            464
CERTI_DIV             0
REG_DATETIME          0
RECT_DATE           175
EMIS_CERTI_NO       188
NOISE_CERTI_NO      624
EMIS_OMIT_NO        624
NOISE_OMIT_NO       624
EMIS_CERTI_DATE     188
NOISE_CERTI_DATE    624
APPLI_ID              0
EF_CD               624
OFFICER_ID          624
PROC_STATE          624
dtype: int64

In [250]:
ct01['CERTI_DIV'].unique()

array([1, 2], dtype=int64)

In [251]:
ct01['EMIS_CERTI_NO'].unique()

array([nan, '19EN*VO*01', '19EN*VO*02', '15EN*CA*03', '18EN*DD*01',
       '17EN*DD*01', '15EN*KJ*02', '18EN*DZ*01', '09EN*HD*01',
       '16EN*DZ*01', '16EN*LK*01', '16EN*SJ*01', '15EN*AP*01',
       '14EN*DD*02', '16EN*TH*01', '16EN*KO*02', '16EN*HD*01',
       '15EN*LS*01', '16EN*KU*03', '16EN*DZ*03', '16EN*DZ*04',
       '16EN*DZ*02', '16EN*HD*02', '16EN*HD*03', '15EN*LS*03',
       '09EN*VO*01', '22EN*AP*01', '12EN*DS*01', '13EN*DS*02',
       '13EN*DS*03', '16EN*HI*01', '16EN*KU*04', '15EN*DZ*02',
       '15EN*KJ*03', '15EN*DZ*03', '16EN*AP*01', '11EN*VO*01',
       '15EN*PE*02', '09EN*DS*02', '17EN*PE*01', '17EN*CU*01',
       '16EN*KO*01', '16EN*KU*01', '09EN*HI*01', '16EN*VO*01',
       '09EN*VO*02', '16EN*KU*02', '09EN*VO*03', '16EN*YA*01',
       '16EN*JD*02', '11EN*VO*04', '19EN*ME*02', '15EN*IV*03',
       '15EN*DS*04', '16EN*HD*04', '16EN*DZ*05', '09EN*CU*06',
       '17EN*LK*01', '15EN*KU*05', '15EN*KU*06', '15EN*KU*13',
       '15EN*YA*04', '17EN*KU*01', '11EN*PE*01', '

In [252]:
len(ct01['EMIS_CERTI_NO'].unique())

426

In [ ]:
# EMIS_VEH_CLASS(배출가스 차종) : CE

In [253]:
temp = ct01.iloc[:, 15:30]
temp.head()

,EMIS_VEH_CLASS,NOISE_VEH_CLASS,EMIS_CTL_INSTALL,EMIS_CTL_INSTALL_E,NOISE_CTL_INSTALL,CO_DEVT,NOX_DEVT,HC_DEVT,HCNOX_DEVT,EVAP_EMIS_DEVT,PM_DEVT,HC_DIV,COLD_CO_DEVT,SMOKE_DEVT,FORMAL_DEVT
0,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
for one in temp.columns:
    print(one, cols_dict[name][one])

EMIS_VEH_CLASS 배출가스 차종
NOISE_VEH_CLASS 소음 차종
EMIS_CTL_INSTALL 배출가스 방지장치 부착내용
EMIS_CTL_INSTALL_E 배출가스 방지장치 부착내용(영문)
NOISE_CTL_INSTALL 소음 방지장치 부착내용
CO_DEVT CO 개발목표치
NOX_DEVT NOx 개발목표치
HC_DEVT HC 개발목표치
HCNOX_DEVT HCNOX 개발목표치
EVAP_EMIS_DEVT 증발가스 개발목표치
PM_DEVT PM 개발목표치
HC_DIV HC 개발목표치
COLD_CO_DEVT Cold CO 개발목표치
SMOKE_DEVT 매연 개발목표치
FORMAL_DEVT 포름알데히드 개발목표치


In [255]:
temp.isnull().sum()

EMIS_VEH_CLASS          0
NOISE_VEH_CLASS       624
EMIS_CTL_INSTALL      624
EMIS_CTL_INSTALL_E    624
NOISE_CTL_INSTALL     624
CO_DEVT               624
NOX_DEVT              624
HC_DEVT               624
HCNOX_DEVT            624
EVAP_EMIS_DEVT        624
PM_DEVT               624
HC_DIV                624
COLD_CO_DEVT          624
SMOKE_DEVT            624
FORMAL_DEVT           624
dtype: int64

In [256]:
ct01.EMIS_VEH_CLASS.unique()

array(['CE'], dtype=object)

In [ ]:
# 

In [257]:
temp = ct01.iloc[:, 30:45]
temp.head()

,DURA_TEST_YN,CERTI_INTRO,CONF_INFO,TEST_PROC,MAINT_WARRANT,EMIS_CTL_LABEL,EMIS_CTL_TECH,EVAP_BLOWBY,AGREE_SUBMIS,EMIS_WARRANT,NOISE_RED_DOC,TEST_FUEL,APP_WAR,CERTI_INTRO_CT,CONF_INFO_CT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
for one in temp.columns:
    print(one, cols_dict[name][one])

DURA_TEST_YN 내구성시험 실시 여부
CERTI_INTRO 인증소개
CONF_INFO 기밀사항
TEST_PROC 시험절차
MAINT_WARRANT 정비보증
EMIS_CTL_LABEL 배출가스 표시판(Label)
EMIS_CTL_TECH 배출가스 제어장치
EVAP_BLOWBY 증발가스 및 브로바이가스
AGREE_SUBMIS 교정정보 및 사후 확정정보 제출 협약
EMIS_WARRANT 자동차배출가스 보증에 관한 제작자의 확인서
NOISE_RED_DOC 자동차 소음저감에 관한 서류
TEST_FUEL 인증시험 연료
APP_WAR 보증에 관한 설명
CERTI_INTRO_CT 인증소개 첨부파일
CONF_INFO_CT 기밀사항 첨부파일


In [259]:
temp.isnull().sum()

DURA_TEST_YN      624
CERTI_INTRO       624
CONF_INFO         624
TEST_PROC         624
MAINT_WARRANT     624
EMIS_CTL_LABEL    624
EMIS_CTL_TECH     624
EVAP_BLOWBY       624
AGREE_SUBMIS      624
EMIS_WARRANT      624
NOISE_RED_DOC     624
TEST_FUEL         624
APP_WAR           624
CERTI_INTRO_CT    624
CONF_INFO_CT      624
dtype: int64

In [ ]:
# 

In [260]:
temp = ct01.iloc[:, 45:60]
temp.head()

,TEST_PROC_CT,MAINT_WARRANT_CT,EMIS_CTL_LABEL_CT,EMIS_CTL_TECH_CT,EVAP_BLOWBY_CT,AGREE_SUBMIS_CT,EMIS_WARRANT_CT,NOISE_RED_DOC_CT,TEST_FUEL_CT,APP_WAR_CT,SIMIL_ADMIN_NO,OMIT_REASON,DF_CO,DF_HC,DF_HCNOX
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
for one in temp.columns:
    print(one, cols_dict[name][one])

TEST_PROC_CT 시험절차 첨부파일 
MAINT_WARRANT_CT 첨부파일
EMIS_CTL_LABEL_CT 배출가스 표시판(Label)첨부파일 
EMIS_CTL_TECH_CT 배출가스 제어장치첨부파일
EVAP_BLOWBY_CT 증발가스 및 브로바이가스첨부파일
AGREE_SUBMIS_CT 교정정보 및 사후 확정정보 제출 협약 첨부파일
EMIS_WARRANT_CT 자동차배출가스 보증에 관한 제작자의 확인서첨부파일
NOISE_RED_DOC_CT 자동차 소음저감에 관한 서류첨부파일
TEST_FUEL_CT 인증시험 연료첨부파일
APP_WAR_CT 보증에 관한 설명첨부파일
SIMIL_ADMIN_NO 인증생략신청서
OMIT_REASON 인증생략근거
DF_CO CO 열화계수
DF_HC HC 열화계수
DF_HCNOX HCNOX 열화계수


In [262]:
temp.isnull().sum()

TEST_PROC_CT         624
MAINT_WARRANT_CT     624
EMIS_CTL_LABEL_CT    624
EMIS_CTL_TECH_CT     624
EVAP_BLOWBY_CT       624
AGREE_SUBMIS_CT      624
EMIS_WARRANT_CT      624
NOISE_RED_DOC_CT     624
TEST_FUEL_CT         624
APP_WAR_CT           624
SIMIL_ADMIN_NO       624
OMIT_REASON          624
DF_CO                624
DF_HC                624
DF_HCNOX             624
dtype: int64

In [ ]:
# 

In [263]:
temp = ct01.iloc[:, 60:75]
temp.head()

,DF_NOX,DF_PM,DF_EVAP_EMIS,DF_COLD_CO,DF_SMOKE,CO_OP,HC_OP,HCNOX_OP,NOX_OP,PM_OP,EVAP_EMIS_OP,COLD_CO_OP,SMOKE_OP,EMIS_TEST_EF_SNO,NOISE_TEST_EF_SNO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
for one in temp.columns:
    print(one, cols_dict[name][one])

DF_NOX NOX 열화계수
DF_PM PM 열화계수
DF_EVAP_EMIS COLD_CO 열화계수
DF_COLD_CO HC 열화계수
DF_SMOKE SMOKE 열화계수
CO_OP 산술기호(1:덧셈 2:곱셈)
HC_OP 산술기호(1:덧셈 2:곱셈)
HCNOX_OP 산술기호(1:덧셈 2:곱셈)
NOX_OP 산술기호(1:덧셈 2:곱셈)
PM_OP 산술기호(1:덧셈 2:곱셈)
EVAP_EMIS_OP 산술기호(1:덧셈 2:곱셈)
COLD_CO_OP Cold CO 연산자
SMOKE_OP 산술기호(1:덧셈 2:곱셈)
EMIS_TEST_EF_SNO 배출가스시험 동일차종 일련번호
NOISE_TEST_EF_SNO 소음시험 동일차종 일련번호


In [265]:
temp.isnull().sum()

DF_NOX               624
DF_PM                624
DF_EVAP_EMIS         624
DF_COLD_CO           624
DF_SMOKE             624
CO_OP                624
HC_OP                624
HCNOX_OP             624
NOX_OP               624
PM_OP                624
EVAP_EMIS_OP         624
COLD_CO_OP           624
SMOKE_OP             624
EMIS_TEST_EF_SNO     624
NOISE_TEST_EF_SNO    624
dtype: int64

In [ ]:
# CERTI_STATE 처분내용 : nan, '90', 'C0', '10', '42', '46', '41'
# RECEIVE_STATE 반영 전 데이타 : nan,  0.

In [267]:
temp = ct01.iloc[:, 75:90]
temp.head()

,DURA_TEST_EF_SNO,ENG_DRAWOUT_YN,OLD_VEH_ID_NO,VEH_ID_NO,CERTI_STATE,RECEIVE_STATE,DURA_TEST_REMARK,EMIS_TEST_REMARK,EF_CD_1,EF_CD_2,EF_CD_3,EF_CD_4,EF_CD_5,EF_CD_6,EF_CD_7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
for one in temp.columns:
    print(one, cols_dict[name][one])

DURA_TEST_EF_SNO 내구성시험 동일차종 일련번호
ENG_DRAWOUT_YN 원동기동력 인출장치 장착여부
OLD_VEH_ID_NO OLD_원동기코드
VEH_ID_NO 원동기코드
CERTI_STATE 처분내용
RECEIVE_STATE 반영 전 데이타
DURA_TEST_REMARK 내구성테스트 비고
EMIS_TEST_REMARK 배출가스시험차 비고
EF_CD_1 인증년도
EF_CD_2 차종기호
EF_CD_3 차종기호
EF_CD_4 차종기호
EF_CD_5 차종기호
EF_CD_6 차종기호
EF_CD_7 차종기호


In [269]:
temp.isnull().sum()

DURA_TEST_EF_SNO    624
ENG_DRAWOUT_YN      624
OLD_VEH_ID_NO       624
VEH_ID_NO           624
CERTI_STATE         174
RECEIVE_STATE       553
DURA_TEST_REMARK    624
EMIS_TEST_REMARK    624
EF_CD_1             624
EF_CD_2             624
EF_CD_3             624
EF_CD_4             624
EF_CD_5             624
EF_CD_6             464
EF_CD_7             624
dtype: int64

In [270]:
ct01.CERTI_STATE.unique()

array([nan, '90', 'C0', '10', '42', '46', '41'], dtype=object)

In [271]:
ct01.RECEIVE_STATE.unique()

array([nan,  0.])

In [ ]:
# NATION 제작국가 : nan, 'SW', 'US', 'KR', 'JP', 'GM', 'IT', 'FN', 'UK', 'CN', 'FR', 'SP', 'SU', 'ID'

In [272]:
temp = ct01.iloc[:, 90:105]
temp.head()

,NATION,TREGREP,TRECOGREP,SEL_CONT,SEL_CONT_E,MODIFY_YN,EMIS_WITHD_DATE,NOISE_WITHD_DATE,OMIT_CNT,RCV_SEQ,PERSON_NO,END_DATE,E_CONFIRM,PAY_TAX,CERTI_TYPE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [273]:
for one in temp.columns:
    print(one, cols_dict[name][one])

NATION 제작국가
TREGREP 정기정비
TRECOGREP 비정기정비
SEL_CONT 비고(한글)
SEL_CONT_E 비고(영문)
MODIFY_YN modify_yn
EMIS_WITHD_DATE 배출인증반납일
NOISE_WITHD_DATE 소음인증반납일
OMIT_CNT 인증생략 가능수
RCV_SEQ RCV순번
PERSON_NO person_no
END_DATE end_date
E_CONFIRM 결제상태
PAY_TAX pay_tax
CERTI_TYPE 인증종류


In [274]:
temp.isnull().sum()

NATION              128
TREGREP             624
TRECOGREP           624
SEL_CONT            624
SEL_CONT_E          624
MODIFY_YN           624
EMIS_WITHD_DATE     605
NOISE_WITHD_DATE    624
OMIT_CNT            624
RCV_SEQ             624
PERSON_NO           624
END_DATE            624
E_CONFIRM           624
PAY_TAX             624
CERTI_TYPE          624
dtype: int64

In [275]:
ct01.NATION.unique()

array([nan, 'SW', 'US', 'KR', 'JP', 'GM', 'IT', 'FN', 'UK', 'CN', 'FR',
       'SP', 'SU', 'ID'], dtype=object)

In [ ]:
# SELF_TEST 시설확인유무 : nan, 'Y', 'N'
# EMIS_CERTI_SNO 배출가스인증번호 : nan,  1.,  2.,  7.,  3.,  5., 12., 13.,  4.,  6., 10.,  8., 11., 9.
# OFFICE_CD 업체코드 : 'DDDC', 'VVIC', 'HIIC', 'KJDC', 'SSIC', 'DSDC', 'HDDC', 'WYIC', 'SJIC', 'ASIC', 'DPIC', 'LSIC', 'KBIC', 'CMIC', 'WTIC', 'MEIC', 'WJIC', 'HCDC', 'TMIC', 'LHIC', 'MXIC', 'HWDC', 'SCIC', 'EXAN', 'RADC', 'TYDC'
# OLDCERTI 과거자료여부 : nan, 'Y'
# REG2_YN 재신청여부 : nan, 'N'

In [276]:
temp = ct01.iloc[:, 105:120]
temp.head()

,SELF_TEST,OBD_TEST_YN,EMIS_NOTE,NOISE_NOTE,EF_CD_8,EF_CD_9,EMIS_CERTI_SNO,NOISE_CERTI_SNO,NOISE_CERTI_CD,OFFICE_CD,USER_ID,OLDCERTI,TONG_DATE,TONG_NUM,REG2_YN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DDDC,5553DDRND1,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DDDC,5553DDRND1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VVIC,5553volvo1,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,VVIC,5553volvo1,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,2.00,NaN,NaN,VVIC,5553volvo1,NaN,NaN,NaN,NaN


In [277]:
for one in temp.columns:
    print(one, cols_dict[name][one])

SELF_TEST 시설확인유무
OBD_TEST_YN OBD 시험여부
EMIS_NOTE 배출허용기준
NOISE_NOTE 소음허용기준
EF_CD_8 차종기호
EF_CD_9 차종기호
EMIS_CERTI_SNO 배출가스인증번호
NOISE_CERTI_SNO 소음인증번호
NOISE_CERTI_CD 소음인증코드
OFFICE_CD 업체코드
USER_ID 사용자아이디
OLDCERTI 과거자료여부
TONG_DATE 통관일
TONG_NUM 통관대수
REG2_YN 재신청여부


In [278]:
temp.isnull().sum()

SELF_TEST          464
OBD_TEST_YN        624
EMIS_NOTE          471
NOISE_NOTE         624
EF_CD_8            624
EF_CD_9            624
EMIS_CERTI_SNO     474
NOISE_CERTI_SNO    624
NOISE_CERTI_CD     624
OFFICE_CD            0
USER_ID            160
OLDCERTI           484
TONG_DATE          624
TONG_NUM           624
REG2_YN            590
dtype: int64

In [279]:
ct01.SELF_TEST.unique()

array([nan, 'Y', 'N'], dtype=object)

In [282]:
ct01.EMIS_CERTI_SNO.unique()

array([nan,  1.,  2.,  7.,  3.,  5., 12., 13.,  4.,  6., 10.,  8., 11.,
        9.])

In [283]:
ct01.OFFICE_CD.unique()

array(['DDDC', 'VVIC', 'HIIC', 'KJDC', 'SSIC', 'DSDC', 'HDDC', 'WYIC',
       'SJIC', 'ASIC', 'DPIC', 'LSIC', 'KBIC', 'CMIC', 'WTIC', 'MEIC',
       'WJIC', 'HCDC', 'TMIC', 'LHIC', 'MXIC', 'HWDC', 'SCIC', 'EXAN',
       'RADC', 'TYDC'], dtype=object)

In [285]:
ct01.OLDCERTI.unique()

array([nan, 'Y'], dtype=object)

In [286]:
ct01.REG2_YN.unique()

array([nan, 'N'], dtype=object)

In [ ]:
# SNOTE 취소반납사유 : nan, '권리 및 의무 승계', '권리 및 의무 승계 신고', '권리 및 의무 신고에 따른 인증서 재교부'
# EF_CNT 동일차량 대수 : nan,  5.,  2.,  1.,  6.,  4.,  3., 22.,  9.,  0., 31.,  8., 14., 23.,  7., 15., 16., 11., 51., 13., 17., 10., 18.

In [287]:
temp = ct01.iloc[:, 120:135]
temp.head()

,SNOTE,DF_MILEAGE,DF_PN,EF_CNT,FILE1,FILE_ORIGIN1,OBD_VEH_CLASS,PRODUCTION,PRODUCTION_E,TC_NO,EV_CTL_INSTALL,ME_RECEIVE_STATE,ME_REFERRAL_SEND_SNO,ME_REFERRAL_RCV_SNO,ME_OPINION_TITLE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,볼보,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,볼보,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
for one in temp.columns:
    print(one, cols_dict[name][one])

SNOTE 취소반납사유
DF_MILEAGE 열화계수 사용량
DF_PN PN 열화계수
EF_CNT 동일차량 대수
FILE1 첨부파일
FILE_ORIGIN1 시설적합확인서
OBD_VEH_CLASS obd_veh_class
PRODUCTION 제작사(한글)
PRODUCTION_E 제작사(영문)
TC_NO 고유번호
EV_CTL_INSTALL 전기제어장치 부착내용
ME_RECEIVE_STATE 환경부 접수 코드(공통코드:21)
ME_REFERRAL_SEND_SNO 검토의뢰서 번호(검토의뢰서 제출번호)
ME_REFERRAL_RCV_SNO 검토의뢰서 접수번호(검토의뢰서 접수번호)
ME_OPINION_TITLE 검토의견서 제목


In [289]:
temp.isnull().sum()

SNOTE                   605
DF_MILEAGE              624
DF_PN                   624
EF_CNT                  464
FILE1                   624
FILE_ORIGIN1            624
OBD_VEH_CLASS           624
PRODUCTION              131
PRODUCTION_E            489
TC_NO                   624
EV_CTL_INSTALL          624
ME_RECEIVE_STATE        624
ME_REFERRAL_SEND_SNO    624
ME_REFERRAL_RCV_SNO     624
ME_OPINION_TITLE        624
dtype: int64

In [291]:
ct01.SNOTE.unique()

array([nan, '권리 및 의무 승계', '권리 및 의무 승계 신고', '권리 및 의무 신고에 따른 인증서 재교부'],
      dtype=object)

In [334]:
set(ct01.EF_CNT.unique())

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 22.0,
 23.0,
 31.0,
 51.0,
 nan}

In [311]:
ct01.loc[ct01.EF_CNT.notnull()].EF_CNT.head()

35    5.00
74    2.00
137   1.00
163   6.00
216   6.00
Name: EF_CNT, dtype: float64

In [ ]:
# ELEC_CAR_GB 전기차 구분 ( N:일반,E:전기차(사용안함),H:하이브리드(사용안함), C:친환경차 ) : 'N', nan
# CONF_ISSUE_NO 발급문서번호 : nan,  821.,  941.,  901., 1141., 1022., 1122.,  822.
# APPROVAL_YN 결재여부 : 'N', 'Y'

In [294]:
temp = ct01.iloc[:, 135:150]
temp.head()

,ME_OPINION_NOTE,ME_OPINION_FILE1,ME_OPINION_FILE2,ME_OPINION_FILE3,ME_OPINION_FILE1_ORGNM,ME_OPINION_FILE2_ORGNM,ME_OPINION_FILE3_ORGNM,ELEC_CAR_GB,CONF_ISSUE_NO,TC_TYPE,APPROVAL_YN,MGR_ID1,MGR_ID2,APPREGI_DATE,SNOTE2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,NaN,NaT,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,NaN,NaT,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,Y,1111john02,1111john02,2019-05-28 11:32:29,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,Y,1111john02,1111john02,2019-05-28 11:32:29,NaN


In [295]:
for one in temp.columns:
    print(one, cols_dict[name][one])

ME_OPINION_NOTE 검토의견서 내용
ME_OPINION_FILE1 검토의견서 파일1
ME_OPINION_FILE2 검토의견서 파일2
ME_OPINION_FILE3 검토의견서 파일3
ME_OPINION_FILE1_ORGNM 검토의견서 파일1(원본파일명)
ME_OPINION_FILE2_ORGNM 검토의견서 파일2(원본파일명)
ME_OPINION_FILE3_ORGNM 검토의견서 파일3(원본파일명)
ELEC_CAR_GB 전기차 구분 ( N:일반,E:전기차(사용안함),H:하이브리드(사용안함), C:친환경차 )
CONF_ISSUE_NO 발급문서번호
TC_TYPE 1:기본인증,2:변경인증
APPROVAL_YN 결재여부
MGR_ID1 결재연구관
MGR_ID2 결재연구사
APPREGI_DATE 결재일
SNOTE2 소음 인증서 반납/취소 사유


In [296]:
temp.isnull().sum()

ME_OPINION_NOTE           624
ME_OPINION_FILE1          624
ME_OPINION_FILE2          624
ME_OPINION_FILE3          624
ME_OPINION_FILE1_ORGNM    624
ME_OPINION_FILE2_ORGNM    624
ME_OPINION_FILE3_ORGNM    624
ELEC_CAR_GB               160
CONF_ISSUE_NO             612
TC_TYPE                   624
APPROVAL_YN                 0
MGR_ID1                   476
MGR_ID2                   616
APPREGI_DATE              476
SNOTE2                    624
dtype: int64

In [297]:
ct01.ELEC_CAR_GB.unique()

array(['N', nan], dtype=object)

In [335]:
set(ct01.CONF_ISSUE_NO.unique())

{821.0, 822.0, 901.0, 941.0, 1122.0, nan, 1022.0, 1141.0}

In [300]:
ct01.TC_TYPE.unique()

array([nan])

In [299]:
ct01.APPROVAL_YN.unique()

array(['N', 'Y'], dtype=object)

In [ ]:
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'C', 'A', 'CA'

In [303]:
temp = ct01.iloc[:, 150:165]
temp.head()

,EMIS_CANCEL_DATE,NOISE_CANCEL_DATE,REG_ID,MOD_ID,MOD_DATETIME,ENG_CAR_GB,EMIS_NOTE2,EMIS_STD_NOTE,NOISE_STD_NOTE
0,NaN,NaN,5553DDRND1,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,5553DDRND1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5553volvo1,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,5553volvo1,1111keun86,20/09/08 10:46:49.000000000,C,"*13-134 엔진코드의 수입사 및 장비명칭, 형식 추가(19.8.27)\r\n*1...",NaN,NaN
4,NaN,NaN,5553volvo1,1111keun86,20/08/05 09:42:20.000000000,C,"*건설기계 제작사 추가 및 13-132, 13-131, 13-133, 13-134,...",NaN,NaN


In [304]:
for one in temp.columns:
    print(one, cols_dict[name][one])

EMIS_CANCEL_DATE 배출가스 인증서 취소 날짜
NOISE_CANCEL_DATE 소음 인증서 취소 날짜
REG_ID 등록아이디
MOD_ID 수정아이디
MOD_DATETIME 수정일시
ENG_CAR_GB 건설기계/농업기계구분
EMIS_NOTE2 변경인증관련비고내용
EMIS_STD_NOTE 배출기준정보
NOISE_STD_NOTE 소음기준정보


In [305]:
temp.isnull().sum()

EMIS_CANCEL_DATE     624
NOISE_CANCEL_DATE    624
REG_ID                 0
MOD_ID               352
MOD_DATETIME         333
ENG_CAR_GB           131
EMIS_NOTE2           363
EMIS_STD_NOTE        619
NOISE_STD_NOTE       624
dtype: int64

In [307]:
ct01.ENG_CAR_GB.unique()

array([nan, 'C', 'A', 'CA'], dtype=object)

In [313]:
ct01.EMIS_NOTE2.unique()

array([nan,
       '*13-134 엔진코드의 수입사 및 장비명칭, 형식 추가(19.8.27)\r\n*16-62, 16-63, 16-64, 16-65 엔진코드의 Dataset변경(부품번호 변경,20.8.4)\r\n*건설기계 형식 추가(20.9.8)',
       '*건설기계 제작사 추가 및 13-132, 13-131, 13-133, 13-134, 13-135, 13-136, 13-137, 13-138 엔진코드의 Dataset변경(부품번호 변경,20.8.4)',
       '*ECM S/W 변경, 수입사 추가, 건설기계 명칭 변경, 형식 추가(19.5.20)\r\n*동일 원동기 추가(20.3.24)\r\n*건설기계 수입사 및 건설기계 형식 추가(20.4.8)\r\n*CE15C18 엔진코드의 정격출력 엔진회전수 변경(20.5.8)\r\n*ECU 및 ECU 소프트웨어 번호 추가(20.11.12)',
       '*17EN*DD*01 전 엔진 DOC 재질 등 변경(17.6.9, 17년7월1일이후 양산 적용)\r\n*4FTA4 전체 엔진의 차압센서 부품번호 추가(18.1.30)\r\n*6424/2584 엔진코드의 건설기계 형식 정정재교부(DSF85G-C, DSF85GB-C&rarr;DSF85C, DSF85CB,18.5.25)\r\n*6426/2684엔진코드의 농업기계 형식 추가(19.12.10)',
       '*전엔진의 연료분사펌프 제작사 등 추가(16.10.21)\r\n*A2300T 엔진의 터보차저 제작사 추가 및 연료분사펌프 연료배분율 등 제원 추가(18.2.1)\r\n*Data Logger 개량에 따른 사양 및 외부형상 변경 동일 적용(18.10.17)\r\n*DPF Controller 개량에 따른 사양 및 외부형상 변경(A2300T5-KT_4 엔진코드의 A5800 농업기계 형식만 해당,19.2.15)\r\n*후처리장치추가(19.4.19)\r\n*농업기계 형식 추가(20.10.23)',
       '*동일원동기 추가(20.3.10)\r\n

In [312]:
ct01.EMIS_STD_NOTE.unique()

array([nan, '*배출가스 허용기준 변경(2020년12월 기준, 22.3.2.9)', '*2015년 1월 기준',
       '2015년 1월기준', '2021년 7월 기준'], dtype=object)

### 2\. UNI_FC_TCERTIMODI

In [314]:
name = name_list[1]
name

'UNI_FC_TCERTIMODI'

In [315]:
ct02 = construct_machine_certification_data_load(name)
ct02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 91 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CERTI_MODI_SNO        2830 non-null   int64         
 1   REG_DATETIME          2830 non-null   object        
 2   CERTI_MODI_DIV        2375 non-null   float64       
 3   EMIS_NOISE_DIV        2670 non-null   float64       
 4   EMIS_MODI_ITEM        0 non-null      float64       
 5   NOISE_MODI_ITEM       0 non-null      float64       
 6   EMIS_MODI_REASON      2346 non-null   object        
 7   NOISE_MODI_REASON     0 non-null      float64       
 8   MODI_ITEM_ETC         0 non-null      float64       
 9   RECT_DATE             2828 non-null   object        
 10  APPLI_ID              2670 non-null   object        
 11  OFFICER_ID            0 non-null      float64       
 12  MODI_EXPLAIN          0 non-null      float64       
 13  EMIS_CHG_EXAM     

#### 유니크 값 탐색

In [ ]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# CERTI_MODI_DIV 변경구분(1:변경인증, 2:변경보고) : 1.,  2., nan
# EMIS_NOISE_DIV 배출소음구분(1:배출 2:소음 3:배출+소음) : 1., nan

In [316]:
temp = ct02.iloc[:, :15]
temp.head()

,CERTI_MODI_SNO,REG_DATETIME,CERTI_MODI_DIV,EMIS_NOISE_DIV,EMIS_MODI_ITEM,NOISE_MODI_ITEM,EMIS_MODI_REASON,NOISE_MODI_REASON,MODI_ITEM_ETC,RECT_DATE,APPLI_ID,OFFICER_ID,MODI_EXPLAIN,EMIS_CHG_EXAM,NOISE_CHG_EXAM
0,4,14/07/31 16:51:43.000000000,1.00,1.00,NaN,NaN,D3400T-007은 트랙터에 탑재되는 원동기로서 당사의 트랙터 라인업을 위하여 시...,NaN,NaN,14/07/31 16:53:35.000000000,5553kmc77,NaN,NaN,NaN,NaN
1,5,15/02/27 19:20:22.000000000,2.00,1.00,NaN,NaN,D3400T-001 원동기의 농업기계 형식 KC87을 추가함,NaN,NaN,15/02/27 19:20:46.000000000,5553kmc77,NaN,NaN,NaN,NaN
2,0,14/08/12 10:00:10.000000000,NaN,1.00,NaN,NaN,NaN,NaN,NaN,14/08/12 11:21:39.000000000,5553wids1,NaN,NaN,NaN,NaN
3,1,14/08/21 13:26:13.000000000,2.00,1.00,NaN,NaN,기존 농업기계 형식의 수입사를 추가하고자 합니다.,NaN,NaN,14/08/21 13:38:18.000000000,5553wids1,NaN,NaN,NaN,NaN
4,2,15/03/18 02:23:09.000000000,2.00,1.00,NaN,NaN,농업기계 수입사의 요청에 따라 기존 동일 원동기의 수입사를 추가하고자 합니다.,NaN,NaN,15/03/18 13:38:07.000000000,5553wids1,NaN,NaN,NaN,NaN


In [317]:
for one in temp.columns:
    print(one, cols_dict[name][one])

CERTI_MODI_SNO 변경인증일련번호
REG_DATETIME 등록일시
CERTI_MODI_DIV 변경구분(1:변경인증, 2:변경보고)
EMIS_NOISE_DIV 배출소음구분(1:배출 2:소음 3:배출+소음)
EMIS_MODI_ITEM 배출가스 변경인증 변경항목
NOISE_MODI_ITEM 소음 변경인증 변경항목
EMIS_MODI_REASON 배출가스 변경사유
NOISE_MODI_REASON 소음 변경사유
MODI_ITEM_ETC 기타 변경사항
RECT_DATE 접수일
APPLI_ID 신청아이디
OFFICER_ID 담당자 ID
MODI_EXPLAIN 변경하고자 하는 인증내용에 대한 설명
EMIS_CHG_EXAM 인증내용 변경전후의 배출가스변화에 대한 설명
NOISE_CHG_EXAM 인증내용 변경전후의 소음변화에 대한 설명


In [318]:
temp.isnull().sum()

CERTI_MODI_SNO          0
REG_DATETIME            0
CERTI_MODI_DIV        455
EMIS_NOISE_DIV        160
EMIS_MODI_ITEM       2830
NOISE_MODI_ITEM      2830
EMIS_MODI_REASON      484
NOISE_MODI_REASON    2830
MODI_ITEM_ETC        2830
RECT_DATE               2
APPLI_ID              160
OFFICER_ID           2830
MODI_EXPLAIN         2830
EMIS_CHG_EXAM        2830
NOISE_CHG_EXAM       2830
dtype: int64

In [336]:
set(ct02.CERTI_MODI_DIV.unique())

{1.0, 2.0, nan}

In [337]:
set(ct02.EMIS_NOISE_DIV.unique())

{1.0, nan}

In [322]:
set(ct02.CERTI_MODI_SNO.unique())

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77}

In [326]:
ct02.EMIS_MODI_ITEM.unique()

array([nan])

In [328]:
ct02.APPLI_ID.unique()

array(['5553kmc77', '5553wids1', '5553kb2058', '5553ls2014', '5553DDRND1',
       '5553PEM01', '5553volvo1', '5553won263', '5553kss980', '5553sks1',
       '5553Doosan', '5553scm01', '5553hic14', '5553jj0323', '5553u01582',
       '5553dpl164', '5553pik305', '5553tmhk14', '5553lmck22',
       '5553HCEC11', '5553yscho1', '5553HWIA01', '5553mest01', nan,
       'EXANex2004', 'VVICVO009', 'RADCEXAN12', 'VVIC5553vo',
       'DDDCkek123', 'WTICwt1095', 'HDDCyej527', 'TYDCtym70',
       'KBICKyh321', 'SCICsks1', 'ASICtkdgh6', 'KBICkb1028'], dtype=object)

In [ ]:
# MODIF_PROC_STATE 변경인증 진행상태 : nan, 0.0
# CERTI_STATE 인증진행상태 : nan, '10', '20', '30', '42', '46', '90', 'C', 'C0', 'C2'

In [323]:
temp = ct02.iloc[:, 15:30]
temp.head()

,MODIF_PROC_STATE,CERTI_STATE,RECEIVE_STATE,CO_STD,NOX_STD,HC_STD,HCNOX_STD,EVAP_EMIS_STD,PM_STD,COLD_CO_STD,SMOKE_STD,FORMAL_STD,IDLE_CO_STD,HC_PPM_STD,MODI_EXPLAIN_CT
0,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [324]:
for one in temp.columns:
    print(one, cols_dict[name][one])

MODIF_PROC_STATE 변경인증 진행상태
CERTI_STATE 인증진행상태
RECEIVE_STATE 반영 전 데이터
CO_STD CO 기준치
NOX_STD NOx 허용기준
HC_STD HC 기준치
HCNOX_STD HCNOX 허용기준
EVAP_EMIS_STD 증발가스 허용기준
PM_STD HC PPm 허용기준
COLD_CO_STD Cold CO 기준치
SMOKE_STD 매연 허용기준
FORMAL_STD FORMAL허용기준
IDLE_CO_STD 아이들링 CO 허용기준
HC_PPM_STD HC PPm 허용기준
MODI_EXPLAIN_CT 변경하고자 하는 인증내용에 대한 설명


In [325]:
temp.isnull().sum()

MODIF_PROC_STATE    1776
CERTI_STATE          254
RECEIVE_STATE       1776
CO_STD              2830
NOX_STD             2830
HC_STD              2830
HCNOX_STD           2830
EVAP_EMIS_STD       2830
PM_STD              2830
COLD_CO_STD         2830
SMOKE_STD           2830
FORMAL_STD          2830
IDLE_CO_STD         2830
HC_PPM_STD          2830
MODI_EXPLAIN_CT     2830
dtype: int64

In [338]:
set(ct02.MODIF_PROC_STATE.unique())

{0.0, nan}

In [330]:
ct02.CERTI_STATE.unique()

array(['90', nan, 'C0', 'C', '42', 'C2', '20', '10', '46', '30'],
      dtype=object)

In [333]:
set(ct02.CERTI_STATE.unique())

{'10', '20', '30', '42', '46', '90', 'C', 'C0', 'C2', nan}

In [ ]:
# 

In [339]:
temp = ct02.iloc[:, 30:45]
temp.head()

,EMIS_CHG_EXAM_CT,NOISE_CHG_EXAM_CT,ACCEL_NOISE_STD,EXH_NOISE_STD,HORN_NOISE_STD,TEST_NO,EMIS_CERTI_DATE,NOISE_CERTI_DATE,MODI_TITLE,MODI_DESC,CERTI_STD,CERTI_STD_DATE,EMIS_WITHD_DATE,NOISE_WITHD_DATE,ADM_NO
0,NaN,NaN,NaN,NaN,NaN,NaN,14/08/11 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52614
1,NaN,NaN,NaN,NaN,NaN,NaN,15/03/13 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52614
2,NaN,NaN,NaN,NaN,NaN,NaN,14/01/13 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52622
3,NaN,NaN,NaN,NaN,NaN,NaN,14/09/02 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52622
4,NaN,NaN,NaN,NaN,NaN,NaN,15/03/27 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52622


In [340]:
for one in temp.columns:
    print(one, cols_dict[name][one])

EMIS_CHG_EXAM_CT 인증내용 변경전후의 배출가스 변화에 대한 설명
NOISE_CHG_EXAM_CT 인증내용 변경전후의 소음 변화에 대한 설명
ACCEL_NOISE_STD 가속주행소음 기준치
EXH_NOISE_STD 배기소음 기준치
HORN_NOISE_STD 경적소음 기준치
TEST_NO 시험번호
EMIS_CERTI_DATE 배출가스인증일 
NOISE_CERTI_DATE 소음인증일
MODI_TITLE 변경제목
MODI_DESC 변경내용
CERTI_STD 인증허용기준
CERTI_STD_DATE 인증허용일
EMIS_WITHD_DATE 배출인증반납일
NOISE_WITHD_DATE 소음인증반납일
ADM_NO 관리번호


In [341]:
temp.isnull().sum()

EMIS_CHG_EXAM_CT     2830
NOISE_CHG_EXAM_CT    2830
ACCEL_NOISE_STD      2830
EXH_NOISE_STD        2830
HORN_NOISE_STD       2830
TEST_NO              2830
EMIS_CERTI_DATE       198
NOISE_CERTI_DATE     2830
MODI_TITLE           2830
MODI_DESC            2830
CERTI_STD            2830
CERTI_STD_DATE       2830
EMIS_WITHD_DATE      2713
NOISE_WITHD_DATE     2830
ADM_NO                  0
dtype: int64

In [344]:
# OLDCERTI 과거자료여부 : nan, 'N', 'Y'
# CERTI_SNO TCERTIMODI의IDX : nan, 1.0 ~ 48.0

In [345]:
temp = ct02.iloc[:, 45:60]
temp.head()

,OLDCERTI,CERTI_SNO,CR_CO,CR_CO2,CR_COLD_CO,CR_EVAP_EMIS,CR_HC,CR_HCNOX,CR_MILEAGE,CR_NOX,CR_PM,CR_PN,MILEAGE_STD,PN_STD,TC_NO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [346]:
for one in temp.columns:
    print(one, cols_dict[name][one])

OLDCERTI 과거자료여부
CERTI_SNO TCERTIMODI의IDX
CR_CO 인증 CO값
CR_CO2 인증 CO2값
CR_COLD_CO 인증 COLD_CO값
CR_EVAP_EMIS 인증 EVAP_EMIS값
CR_HC 인증 HC값
CR_HCNOX 인증 HCNOX값
CR_MILEAGE 인증 MILEAGE값
CR_NOX 인증 NOX값
CR_PM 인증 PM값
CR_PN 인증 PN값
MILEAGE_STD 허용기준MILEAGE
PN_STD PN 허용기준
TC_NO 고유번호


In [347]:
temp.isnull().sum()

OLDCERTI        2211
CERTI_SNO       1778
CR_CO           2830
CR_CO2          2830
CR_COLD_CO      2830
CR_EVAP_EMIS    2830
CR_HC           2830
CR_HCNOX        2830
CR_MILEAGE      2830
CR_NOX          2830
CR_PM           2830
CR_PN           2830
MILEAGE_STD     2830
PN_STD          2830
TC_NO           2830
dtype: int64

In [349]:
set(ct02.OLDCERTI.unique())

{'N', 'Y', nan}

In [350]:
set(ct02.CERTI_SNO.unique())

{1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 nan}

In [ ]:
# EMIS_DOC 배출가스인증서교부 공문 내용 : 1.0, nan
# CERTI_DIV 인증구분(1:국내,2:수입) : 1.0, 2.0, nan
# APPROVAL_YN 결재여부 : 'N', 'Y'
# MAKE_MTRL_NO 인증신청 작성자료번호 : 1.0, 3.0, 4.0, 5.0, 6.0, nan

In [351]:
temp = ct02.iloc[:, 60:75]
temp.head()

,OPINION_NOTE,BOWAN_NOTE,CTEST_NOTE,EMIS_DOC,NOISE_DOC,CERTI_DIV,APPROVAL_YN,MGR_ID1,MGR_ID2,APPREGI_DATE,MAKE_MTRL_NO,EMIS_CANCEL_DATE,NOISE_CANCEL_DATE,SNOTE,SNOTE2
0,NaN,NaN,NaN,NaN,NaN,1.00,N,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.00,N,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,2.00,N,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,2.00,N,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.00,N,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [352]:
for one in temp.columns:
    print(one, cols_dict[name][one])

OPINION_NOTE 최종의견
BOWAN_NOTE 보완사항
CTEST_NOTE 확인시험
EMIS_DOC 배출가스인증서교부 공문 내용
NOISE_DOC 소음인증서교부 공문 내용
CERTI_DIV 인증구분(1:국내,2:수입)
APPROVAL_YN 결재여부
MGR_ID1 결재연구관
MGR_ID2 결재연구사
APPREGI_DATE 결재일
MAKE_MTRL_NO 인증신청 작성자료번호
EMIS_CANCEL_DATE 배출가스 인증서 취소 날짜
NOISE_CANCEL_DATE 소음 인증서 취소 날짜
SNOTE 배출가스 인증서 취소 사유
SNOTE2 소음 인증서 취소 사유


In [353]:
temp.isnull().sum()

OPINION_NOTE         2031
BOWAN_NOTE           2830
CTEST_NOTE           2830
EMIS_DOC             1798
NOISE_DOC            2830
CERTI_DIV             160
APPROVAL_YN             0
MGR_ID1              1814
MGR_ID2              2736
APPREGI_DATE         1814
MAKE_MTRL_NO         1614
EMIS_CANCEL_DATE     2830
NOISE_CANCEL_DATE    2830
SNOTE                2830
SNOTE2               2830
dtype: int64

In [355]:
set(ct02.EMIS_DOC.unique())

{1.0, nan}

In [357]:
set(ct02.CERTI_DIV.unique())

{1.0, 2.0, nan}

In [358]:
set(ct02.APPROVAL_YN.unique())

{'N', 'Y'}

In [359]:
set(ct02.MAKE_MTRL_NO.unique())

{1.0, 3.0, 4.0, 5.0, 6.0, nan}

In [ ]:
# STANDARD_SLT 배출허용기준 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# STANDARD_SLT2 배출허용기준2 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# EMIS_ROWS_CODE 배출가스저감기술 : nan, '1', '1/2', '1/2/3', '1/2/3/4', '1/2/3/4/5', '1/2/4', '1/2/4/5', '1/3', '1/3/4', '1/3/4/5', '1/4', '1/4/5', '1/5', '2', '2/3/4', '2/4', '3/4', '5'
# EMIS_LOWS 배출가스저감기술코드 : nan, 'In-cylinder 방식', 'LNC', 'Turbocharger, Common rail', 'VGT, common rail', '연료장치, 터보챠저', '전자제어 연료분사 시스템', '전자제어 연료분사시스템', '전자제어장치,커먼레일', '커먼레일', '터보', '터보, 기계식', '터보차저', '터보차져'

In [360]:
temp = ct02.iloc[:, 75:90]
temp.head()

,REG_ID,MOD_ID,MOD_DATETIME,STANDARD_SLT,STANDARD_SLT2,ETC_TXT1,IDX,ETC_TXT3,ETC_TXT4,EMIS_ROWS_CODE,EMIS_LOWS,PHP_CERTY_TYPE,PHP_REP_TYPE,PHP_CERTI_STATE,OLD_EMIS_CERTI_DATE
0,NaN,NaN,NaN,NaN,NaN,실린더 헤드커버 상면,495.00,3.5,20,5,NaN,2.00,4.00,8.00,NaN
1,NaN,NaN,NaN,NaN,NaN,실린더 헤드커버 상면,757.00,3.5,20,5,NaN,2.00,4.00,8.00,NaN
2,NaN,NaN,NaN,3.00,NaN,엔진 기어 케이스 상단부,502.00,3.0,15.0,1/3/4/5,"VGT, common rail",4.00,4.00,8.00,NaN
3,NaN,NaN,NaN,NaN,NaN,엔진 기어 케이스 상단부,517.00,3.0,15.0,1/3/4/5,NaN,2.00,4.00,8.00,NaN
4,NaN,NaN,NaN,NaN,NaN,엔진 기어 케이스 상단부,774.00,3.0,15.0,1/3/4/5,NaN,2.00,4.00,8.00,NaN


In [361]:
for one in temp.columns:
    print(one, cols_dict[name][one])

REG_ID 등록아이디
MOD_ID 수정아이디
MOD_DATETIME 수정일시
STANDARD_SLT 배출허용기준
STANDARD_SLT2 배출허용기준2
ETC_TXT1 배출가스표지판위치
IDX PHP건설기계IDX
ETC_TXT3 최대부압
ETC_TXT4 최대배압
EMIS_ROWS_CODE 배출가스저감기술
EMIS_LOWS 배출가스저감기술코드
PHP_CERTY_TYPE 인증_형태(이전PHP사용컬럼)
PHP_REP_TYPE 접수_형태(이전PHP사용컬럼)
PHP_CERTI_STATE 인증_상태(이전PHP사용컬럼)
OLD_EMIS_CERTI_DATE 과거자료 배출인증일자


In [362]:
temp.isnull().sum()

REG_ID                 1721
MOD_ID                 1917
MOD_DATETIME           1531
STANDARD_SLT           2288
STANDARD_SLT2          2717
ETC_TXT1               1054
IDX                    1214
ETC_TXT3               1054
ETC_TXT4               1087
EMIS_ROWS_CODE         1755
EMIS_LOWS              2757
PHP_CERTY_TYPE         1214
PHP_REP_TYPE           1214
PHP_CERTI_STATE        1214
OLD_EMIS_CERTI_DATE    2830
dtype: int64

In [364]:
set(ct02.STANDARD_SLT.unique())

{1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0, nan}

In [365]:
set(ct02.STANDARD_SLT2.unique())

{1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, nan, 9.0, 10.0}

In [367]:
set(ct02.EMIS_ROWS_CODE.unique())

{'1',
 '1/2',
 '1/2/3',
 '1/2/3/4',
 '1/2/3/4/5',
 '1/2/4',
 '1/2/4/5',
 '1/3',
 '1/3/4',
 '1/3/4/5',
 '1/4',
 '1/4/5',
 '1/5',
 '2',
 '2/3/4',
 '2/4',
 '3/4',
 '5',
 nan}

In [368]:
set(ct02.EMIS_LOWS.unique())

{'In-cylinder 방식',
 'LNC',
 'Turbocharger, Common rail',
 'VGT, common rail',
 nan,
 '연료장치, 터보챠저',
 '전자제어 연료분사 시스템',
 '전자제어 연료분사시스템',
 '전자제어장치,커먼레일',
 '커먼레일',
 '터보',
 '터보, 기계식',
 '터보차저',
 '터보차져'}

In [ ]:
#

In [369]:
temp = ct02.iloc[:, 90:105]
temp.head()

,OLD_NOISE_CERTI_DATE
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [370]:
for one in temp.columns:
    print(one, cols_dict[name][one])

OLD_NOISE_CERTI_DATE 과거자료 소음인증일자


In [371]:
temp.isnull().sum()

OLD_NOISE_CERTI_DATE    2830
dtype: int64

### 3\. UNI_FC_ENG_TEFSPEC

In [373]:
name = name_list[2]
name

'UNI_FC_ENG_TEFSPEC'

In [374]:
ct03 = construct_machine_certification_data_load(name)
ct03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8646 entries, 0 to 8645
Columns: 108 entries, ADM_NO to MAX_OUTT_POWER_UNIT
dtypes: float64(23), int64(3), object(82)
memory usage: 7.1+ MB


#### 유니크 값 탐색

In [ ]:
# 

In [375]:
temp = ct03.iloc[:, :15]
temp.head()

,ADM_NO,CERTI_MODI_SNO,EF_SNO,ENG_MAKER,ENG_TYPE,ENG_ID_NO,CYCLE,BOAR,STROKE,CYLIN_CNT,CYLIN_ARRANGE,ENG_DISPL,RATED_OUT,RATED_OUT_RPM,MAX_TORQ
0,52985,0,5,구보다(KUBOTA),D722-E4,D722-E4-KR02,NaN,NaN,NaN,NaN,NaN,719.00,10.2,"2,500",NaN
1,52985,2,7,구보다(KUBOTA),D722-E4,D722-E4-KR00,NaN,NaN,NaN,NaN,NaN,719.00,14.7,"3,600",NaN
2,52985,3,9,구보다(KUBOTA),D722-E4,D722-E4-KR01,NaN,NaN,NaN,NaN,NaN,719.00,7.5,"2,000",NaN
3,52985,3,8,구보다(KUBOTA),D722-E4,D722-E4-KR01,NaN,NaN,NaN,NaN,NaN,719.00,7.5,"2,000",NaN
4,52985,4,13,구보다(KUBOTA),D722-E4,D722-E4-KR02,NaN,NaN,NaN,NaN,NaN,719.00,10.2,"2,500",NaN


In [376]:
for one in temp.columns:
    print(one, cols_dict[name][one])

ADM_NO 관리번호
CERTI_MODI_SNO 변경인증일련번호
EF_SNO 동일원동기일련번호
ENG_MAKER 제작사
ENG_TYPE 원동기 형식
ENG_ID_NO 원동기코드
CYCLE 싸이클(행정)
BOAR 보어(mm)
STROKE 스트로크(mm)
CYLIN_CNT 실린더 개수
CYLIN_ARRANGE 실린더 배치
ENG_DISPL 배기량(㎠)
RATED_OUT 정격출력(kW)
RATED_OUT_RPM 회전속도(rpm)
MAX_TORQ 최대토크(Nm)


In [377]:
temp.isnull().sum()

ADM_NO               0
CERTI_MODI_SNO       0
EF_SNO               0
ENG_MAKER           84
ENG_TYPE             1
ENG_ID_NO            1
CYCLE             5101
BOAR              6454
STROKE            6454
CYLIN_CNT         5004
CYLIN_ARRANGE     6501
ENG_DISPL            2
RATED_OUT            1
RATED_OUT_RPM        1
MAX_TORQ          4997
dtype: int64

In [401]:
set(ct03.ENG_CAR_GB.unique())

{'A', 'C', 'CA', nan}

### 4\. UNI_FC_TEEMISSN

In [402]:
name = name_list[3]
name

'UNI_FC_TEEMISSN'

In [403]:
ct04 = construct_machine_certification_data_load(name)
ct04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           1058 non-null   int64  
 1   CERTI_MODI_SNO   1058 non-null   int64  
 2   FUEL             882 non-null    object 
 3   SEL_STAND        592 non-null    float64
 4   F_STAND          609 non-null    float64
 5   GU_TERM          577 non-null    float64
 6   TEST_DIV         0 non-null      float64
 7   TEST_METHOD_CHK  1053 non-null   object 
 8   COL              0 non-null      float64
 9   REG_ID           1058 non-null   object 
 10  REG_DATETIME     1058 non-null   object 
 11  MOD_ID           538 non-null    object 
 12  MOD_DATETIME     538 non-null    object 
 13  TBL_PK_SEQ_ID    1058 non-null   int64  
 14  LEV_CRTR_CD      0 non-null      float64
dtypes: float64(6), int64(3), object(6)
memory usage: 124.1+ KB


In [412]:
set(ct04.LEV_CRTR_CD.unique())

{nan}

### 5\. UNI_FC_TEEMISSN_ENG_RES

In [413]:
name = name_list[4]
name

'UNI_FC_TEEMISSN_ENG_RES'

In [414]:
ct05 = construct_machine_certification_data_load(name)
ct05.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2761 entries, 0 to 2760
Columns: 101 entries, ADM_NO to NH3_ST_PERMSS
dtypes: float64(77), int64(5), object(19)
memory usage: 2.1+ MB


In [421]:
set(ct05.TEST_NO.unique())

{' A3710273',
 ' A4710537, A4710538',
 ' C4710286',
 '-',
 '0802-001',
 '1',
 '1006701',
 '1007736',
 '161.08.1197.01',
 '2',
 '20190926-F-11-002',
 '20190926-F11-001',
 '20200521',
 '20200522',
 '2021-07-02',
 '20210702',
 '20210726-F07-030',
 '20210727-F07-033',
 '20210730-F11-001',
 '20210813-01',
 '20210813-02',
 '20210825-01',
 '20210826-01',
 '20210913-01',
 '20210913-02',
 '20211124-E04-001',
 '20211124-E04-002',
 '20211228_ECE_NRTC_1',
 '20211229_ECE_NRSC_1',
 '20220107_ECE_NRTC_1',
 '20220113-ECE_NRSC_1',
 '20220125-F11-001 NRTC',
 '20220125-F11-002 RMC',
 '20220126_ECE_NRTC_1',
 '20220127_ECE_NRSC_1',
 '20220209-E13-001',
 '20220209-E13-002',
 '20220316-F11-001 NRTC',
 '20220316-F11-002 RMC',
 '20220922-F11-001 NRTC',
 '20220922-F11-002 RMC',
 '20230131-F11-001_NRTC',
 '20230131-F11-002 RMC',
 '210422_3A165 1821-2213E RMC Test Report v01',
 '210702_C1',
 '22-20210929-E13-001k',
 '4344_RN023B.TDL',
 '4344_TN023D.TDL',
 '62295007 .1',
 '62295011 .1',
 '69800673 .2',
 '69800678 

### 6\. UNI_FC_TESTINFO

In [422]:
name = name_list[5]
name

'UNI_FC_TESTINFO'

In [423]:
ct06 = construct_machine_certification_data_load(name)
ct06.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ADM_NO                         1044 non-null   int64  
 1   EF_SNO                         0 non-null      float64
 2   CERTI_MODI_SNO                 1044 non-null   int64  
 3   INFO1                          177 non-null    object 
 4   INFO2                          1044 non-null   object 
 5   INFO3                          0 non-null      float64
 6   TC_TYPE                        1044 non-null   int64  
 7   INFO4                          0 non-null      float64
 8   ELEC_INFO1                     0 non-null      float64
 9   ELEC_TECH                      0 non-null      float64
 10  ELEC_INFO1_TXT                 0 non-null      float64
 11  ELEC_WARR_ACCUMU               0 non-null      float64
 12  ELEC_WARR_CHARGER              0 non-null      f

In [429]:
set(ct06.INFO4.unique())

{nan}

### 7\. UNI_FC_ENG_LIST

In [430]:
name = name_list[6]
name

'UNI_FC_ENG_LIST'

In [431]:
ct07 = construct_machine_certification_data_load(name)
ct07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9521 entries, 0 to 9520
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ENG_SNO          9521 non-null   int64  
 1   ADM_NO           9521 non-null   int64  
 2   CERTI_MODI_SNO   9521 non-null   int64  
 3   EF_SNO           9521 non-null   int64  
 4   ENG_TYPE         5 non-null      object 
 5   ENG_ID_NO        5 non-null      object 
 6   MAX_PWR_KW       5 non-null      float64
 7   MAX_PWR_RPM      5 non-null      float64
 8   ENG_DISPL        5 non-null      float64
 9   VEH_IN_COM       9281 non-null   object 
 10  VEH_PRE_NM       9279 non-null   object 
 11  VEH_NM           9287 non-null   object 
 12  VEH_MAKER        9291 non-null   object 
 13  VEH_TYPE         9290 non-null   object 
 14  ETC              58 non-null     object 
 15  REG_DATETIME     9521 non-null   object 
 16  REG_ID           2986 non-null   object 
 17  MOD_DATETIME  

In [437]:
set(ct07.VEH_TYPE_YN.unique())

{'Y', nan}

### 8\. UNI_FC_ENDUENG_EMI

In [438]:
name = name_list[7]
name

'UNI_FC_ENDUENG_EMI'

In [439]:
ct08 = construct_machine_certification_data_load(name)
ct08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           411 non-null    int64  
 1   CERTI_MODI_SNO   411 non-null    int64  
 2   TEST_METHOD_CHK  411 non-null    object 
 3   TEST_MOD         411 non-null    int64  
 4   TEST_RST_SNO     411 non-null    int64  
 5   NOX_GRAD         167 non-null    float64
 6   NMHC_GRAD        153 non-null    float64
 7   CO_GRAD          163 non-null    float64
 8   PM_GRAD          166 non-null    float64
 9   NOX_YITCT        166 non-null    float64
 10  NMHC_YITCT       153 non-null    float64
 11  CO_YITCT         165 non-null    float64
 12  PM_YITCT         165 non-null    float64
 13  REG_ID           411 non-null    object 
 14  REG_DATETIME     375 non-null    object 
 15  MOD_ID           57 non-null     object 
 16  MOD_DATETIME     21 non-null     object 
 17  PN_GRAD         

In [444]:
set(ct08.TEST_RST_SNO.unique())

{1}

### 9\. UNI_FC_ENDUENGTEST

In [445]:
name = name_list[8]
name

'UNI_FC_ENDUENGTEST'

In [446]:
ct09 = construct_machine_certification_data_load(name)
ct09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          430 non-null    int64 
 1   CERTI_MODI_SNO  430 non-null    int64 
 2   TEST_ENG_SNO    430 non-null    int64 
 3   ENG_TYPE        334 non-null    object
 4   ENG_ID_NO       315 non-null    object
 5   ENG_NO          319 non-null    object
 6   TEST_NO         277 non-null    object
 7   TEST_DATE       318 non-null    object
 8   REG_ID          407 non-null    object
 9   REG_DATETIME    430 non-null    object
 10  MOD_ID          56 non-null     object
 11  MOD_DATETIME    51 non-null     object
dtypes: int64(3), object(9)
memory usage: 40.4+ KB


In [453]:
set(ct09.ENG_TYPE.unique())

{'*KBXL01.3DCB',
 '*KBXL01.7ECB',
 '*KBXL03.3E2D',
 '1206F-E70TTA',
 '1260F-E70TTA',
 '3029HPRNT4',
 '3A165',
 '3C100-D4',
 '3F183T',
 '3H5',
 '4045HPRNT17',
 '4045HPRNT18',
 '4HTA-D4',
 '4HTI-D4',
 '4HTI5',
 '4JJ1XDDV',
 '4JTA-D4',
 '4JTA5',
 '4TN107FTT',
 '904J-E36TA',
 'A2300',
 'A2400T',
 'C18',
 'C9.3B',
 'D3400T',
 'D956 A7-05',
 'DM03V',
 'DM03V ',
 'ED22',
 'F4HFE6134B',
 'F4HGE613I',
 'F5BFL413E',
 'F5HFL464C',
 'F5HGL413F',
 'HKBXL02.2ECB',
 'ISB',
 'KKBXL03.3E2D (EU StageV)',
 'L4C25-T1',
 'L9',
 'LKBXL06.1A5D',
 'OM473LA',
 'PKBXL05.0C5D (EU StageV)',
 'PKBXL05.0C5D(EU StageV)',
 'SAA6D140E-7',
 'TAD1375 VE\t\t',
 'TAD873VE\t\t',
 'TCD16.0 V8',
 'TCD3.6L4',
 nan}

### 10\. UNI_FC_ENDUENGTEST_RES

In [454]:
name = name_list[9]
name

'UNI_FC_ENDUENGTEST_RES'

In [ ]:
ct10 = construct_machine_certification_data_load(name)
ct10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            7581 non-null   int64  
 1   CERTI_MODI_SNO    7581 non-null   int64  
 2   P_ENG_YN          7581 non-null   int64  
 3   SEL_ADM_NO        4236 non-null   float64
 4   TEST_METHOD_CHK   7581 non-null   object 
 5   TEST_MOD          7581 non-null   int64  
 6   TEST_RST_SNO      7581 non-null   int64  
 7   TEST_RST_SNO_SEQ  7581 non-null   int64  
 8   FUEL              0 non-null      float64
 9   DDC_USE_YN        6724 non-null   float64
 10  TEST_CNT          0 non-null      float64
 11  TEST_TIME         7100 non-null   float64
 12  TEST_PLACE        7054 non-null   object 
 13  TEST_DATE         7025 non-null   object 
 14  CO                6034 non-null   float64
 15  NOX               5960 non-null   float64
 16  PM                5901 non-null   float64


In [ ]:
set(ct10.TEST_CNT.unique())

{nan}

### 11\. UNI_FC_KIENGTEST

In [472]:
name = name_list[10]
name

'UNI_FC_KIENGTEST'

In [473]:
ct11 = construct_machine_certification_data_load(name)
ct11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          307 non-null    int64 
 1   CERTI_MODI_SNO  307 non-null    int64 
 2   TEST_ENG_SNO    307 non-null    int64 
 3   ENG_TYPE        209 non-null    object
 4   ENG_ID_NO       196 non-null    object
 5   ENG_NO          209 non-null    object
 6   TEST_NO         179 non-null    object
 7   TEST_DATE       203 non-null    object
 8   REG_ID          307 non-null    object
 9   REG_DATETIME    307 non-null    object
 10  MOD_ID          29 non-null     object
 11  MOD_DATETIME    29 non-null     object
dtypes: int64(3), object(9)
memory usage: 28.9+ KB


In [478]:
set(ct11.TEST_NO.unique())

{'16_2399_ENRTC227_CW',
 '20210812-01',
 '20210812-02',
 '20210830-01',
 '20210830-02',
 '20210907-03',
 '2102305 (1차)',
 '2102305 (2차)',
 '2102405 (1차)',
 '2102405 (2차)',
 '210630_N2',
 '210701_N1',
 '210705_N1',
 '217605 (1차)',
 '217605 (2차)',
 '3',
 '3,4,5',
 '3-1 재생이 발생하지 않는 조건',
 '3-1. 재생이 발생 하지 않는 조건 1차',
 '3-1. 재생이 발생 하지 않는 조건 2차',
 '3-1. 재생이 발생하지 않는 조건 #1 재생 전',
 '3-1. 재생이 발생하지 않는 조건 1차',
 '3-2 재생이 발생하지 않는 조건',
 '3-2. 재생이 발생 하지 않는 조건 2차',
 '3-2. 재생이 발생하지 않는 조건 #2 재생 후',
 '3-2. 재생이 발생하지 않는 조건 2차',
 '3/4/5',
 '31005181, 31005183, 31005184',
 '31005445, 31005446, 31005447',
 '31005468, 31005469, 31005470',
 '31005517, 31005518, 31005519',
 '31005533, 31005534, 31005535',
 '4-1 재생 조건',
 '4-1. 재생 조건 #1',
 '4-1. 재생 조건 (Assist 재생 1차)',
 '4-1. 재생 조건 (Assist 재생)',
 '4-1. 재생 조건_Assist 재생 1차',
 '4-2. Reset 재생 조건 1차',
 '4-2. 재생 조건 #2',
 '4-2. 재생 조건 (Assist 재생 2차)',
 '4-2. 재생 조건 (Reset 재생 1차)',
 '4-2. 재생 조건 (Reset 재생)',
 '4-2. 재생 조건 1차',
 '4-2. 재생 조건_Assist 재생 2차',
 '4-3. Reset 재생 조건 2차',
 

### 12\. UNI_FC_KIENGTEST_RES

In [479]:
name = name_list[11]
name

'UNI_FC_KIENGTEST_RES'

In [480]:
ct12 = construct_machine_certification_data_load(name)
ct12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [489]:
set(ct12.FUEL.unique())

{nan}

## 과제2 > 건설기계 인증대장

In [126]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld34 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='TIER-3~TIER-4')
cert_ld34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5974 entries, 0 to 5973
Data columns (total 64 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   배출기준             5971 non-null   object
 1   상호명              5971 non-null   object
 2   인증번호             5971 non-null   object
 3   최초인증일자           5971 non-null   object
 4   재교부일자
(변경일자)     5971 non-null   object
 5   접수번호             5971 non-null   object
 6   접수일자             5971 non-null   object
 7   종류               5971 non-null   object
 8   구분               5971 non-null   object
 9   상세내역             4051 non-null   object
 10  엔진형식
(엔진모델명)     5971 non-null   object
 11  엔진코드             5970 non-null   object
 12  엔진배기량            5862 non-null   object
 13  최대출력             5856 non-null   object
 14  Unnamed: 14      5856 non-null   object
 15  엔진제작사            5971 non-null   object
 16  엔진제작국            5971 non-null   object
 17  건설기계 
수입사
(대표자)  5367 non-null   

In [161]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld5 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='STAGE5')
cert_ld5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2058 entries, 0 to 2057
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   배출기준             2054 non-null   object 
 1   상호명              2055 non-null   object 
 2   인증번호             2055 non-null   object 
 3   최초인증일자           2054 non-null   object 
 4   재교부일자
(변경일자)     2054 non-null   object 
 5   접수번호             2054 non-null   object 
 6   접수일자             2054 non-null   object 
 7   종류               2054 non-null   object 
 8   구분               2054 non-null   object 
 9   상세내역             783 non-null    object 
 10  엔진형식
(엔진모델명)     2054 non-null   object 
 11  엔진코드             2053 non-null   object 
 12  엔진배기량            2054 non-null   float64
 13  최대출력             2050 non-null   object 
 14  Unnamed: 14      2050 non-null   object 
 15  엔진
제작사           2054 non-null   object 
 16  엔진제작국            2054 non-null   object 
 17  건설기계 
수입사
(대표자

### 정보(TIER-3~TIER-4)

#### 컬럼 수정

In [128]:
cert_ld34.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '정정내역', '기준치', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', '자체시험여부', '적용기술', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'DF 변경보고'],
      dtype='object')

In [129]:
cert_ld34.iloc[:5, :20]

,배출기준,상호명,인증번호,최초인증일자,재교부일자\n(변경일자),접수번호,접수일자,종류,구분,상세내역,엔진형식\n(엔진모델명),엔진코드,엔진배기량,최대출력,Unnamed: 14,엔진제작사,엔진제작국,건설기계 \n수입사\n(대표자),건설기계\n제작사,건설기계\n명칭
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KW,RPM,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,2009-01-14 00:00:00,-,6940,2008-12-23 00:00:00,건설기계,대표(최초),NaN,QSM,20021,10823,250\n(325ps),"2,100\n(1,900)",커민스,미국,국내,현대중공업,굴삭기
4,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-02-08 00:00:00,15,2012-02-06 00:00:00,건설기계,대표(정정),대표엔진의\n건설기계 형식 정정,QSM,20021,10823,250\n(325ps),"2,100\n(1,900)",커민스,미국,댕기상사㈜\n(김학희),C.V.S,지게차


In [131]:
cert_ld34.iloc[:5, 20:40]

,건설/농업기계 형식,비고,정정내역,기준치,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,시험결과,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,NaN,NaN,NaN,CO,NMHC+Nox,NMHC,Nox,PM,CO2,ISO 8178 C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NRSC,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CO,HC+Nox,NMHC,Nox,PM,CO2,검사일시,최대출력,NaN,CO,HC+Nox
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kw,rpm,NaN,NaN
3,"ROBEX4700LC-7A, ROBEX4700-7, ROBEX5000LC-7, RO...",NaN,"1. 동일엔진 추가, 출력병기\n (2010.02.02, 재교부)",3.50,4,-,-,0.20,-,1.32,3.70,-,-,0.17,776,2005-12-14 00:00:00,340ps,2100,-,-
4,F477.5,NaN,"1.건설기계 형식 정정(2012.02.,재교부)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
cert_ld34.iloc[:5, 40:60]

,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,자체시험여부,적용기술,Unnamed: 58,Unnamed: 59
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NRTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EGR,SCR,DPF
1,NMHC,Nox,PM,CO2,검사일시,최대출력,NaN,CO,HC+Nox,NMHC,Nox,PM,CO2,검사일시,최대출력,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,kw,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kw,rpm,NaN,NaN,NaN,NaN
3,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,"TIER-2기준으로시험,3도만족\n교통입회",X,X,X
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
cert_ld34.iloc[:5, 60:]

,Unnamed: 60,Unnamed: 61,Unnamed: 62,DF 변경보고
0,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,O,NaN,NaN,해당없음
4,NaN,NaN,NaN,해당없음


In [136]:
cert_ld34.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', '최대출력_KW', '최대출력_RPM', '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', '정정내역', 
    '기준치_CO', '기준치_NMHC+Nox', '기준치_NMHC', '기준치_Nox', '기준치_PM', '기준치_CO2', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_HC+Nox', '시험결과_ISO_8178_C1_NMHC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_검사일시', '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_HC+Nox', '시험결과_NRSC_NMHC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_CO2', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_HC+Nox', '시험결과_NRTC_NMHC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_CO2', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm',
    '자체시험여부', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 'DF_변경보고'
]
len(cert_ld34.columns)

64

#### 필요한 데이터만 추출(행 제한)

In [137]:
cert_ld34 = cert_ld34.iloc[3:, :]
cert_ld34.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,DF_변경보고
3,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,2009-01-14 00:00:00,-,6940,2008-12-23 00:00:00,건설기계,대표(최초),NaN,...,-,-,"TIER-2기준으로시험,3도만족\n교통입회",X,X,X,O,NaN,NaN,해당없음
4,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-02-08 00:00:00,15,2012-02-06 00:00:00,건설기계,대표(정정),대표엔진의\n건설기계 형식 정정,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
5,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-08-03 00:00:00,173,2011-07-26 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
6,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-09-22 00:00:00,204,2011-09-07 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
7,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-01-25 00:00:00,3,2012-01-10 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [184]:
cert_ld34.shape

(5971, 64)

### 데이터 확인

In [509]:
set(cert_ld34['DF_변경보고'].unique())

{'16년부터 수입 안함', 2200, 'O', nan, '신규인증완료후 미생산', '해당없음'}

### 데이터 수정(TIER-3~TIER-4)

In [ ]:
# 줄 바꿈이 있는 열
# 상세내역, 최대출력_KW, 최대출력_RPM, 정정내역, 자체시험여부

# 줄 바꿈을 간격으로 바꿀 열
# 상세내역

# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역

# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부

In [141]:
# 줄 바꿈을 간격으로 바꿀 열
# 상세내역
cert_ld34['상세내역'] = cert_ld34['상세내역'].str.replace('\n', ' ')
cert_ld34[['상세내역']].head()

,상세내역
3,NaN
4,대표엔진의 건설기계 형식 정정
5,대표엔진의 수입사 및 건설기계추가
6,대표엔진의 수입사 및 건설기계추가
7,대표엔진의 수입사 및 건설기계추가


In [140]:
# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역
cert_ld34['최대출력_KW'] = cert_ld34['최대출력_KW'].str.replace('\n', '')
cert_ld34['최대출력_RPM'] = cert_ld34['최대출력_RPM'].str.replace('\n', '')
cert_ld34['정정내역'] = cert_ld34['정정내역'].str.replace('\n', '')
cert_ld34[['최대출력_KW', '최대출력_RPM', '정정내역']].head()

,최대출력_KW,최대출력_RPM,정정내역
3,250(325ps),"2,100(1,900)","1. 동일엔진 추가, 출력병기 (2010.02.02, 재교부)"
4,250(325ps),"2,100(1,900)","1.건설기계 형식 정정(2012.02.,재교부)"
5,250(325ps),"2,100(1,900)",NaN
6,250(325ps),"2,100(1,900)",NaN
7,250(325ps),"2,100(1,900)",NaN


In [142]:
# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부
cert_ld34['자체시험여부'] = cert_ld34['자체시험여부'].str.replace('\n', ',')
cert_ld34[['자체시험여부']].head()

,자체시험여부
3,"TIER-2기준으로시험,3도만족,교통입회"
4,NaN
5,NaN
6,NaN
7,NaN


### 정보(STAGE5)

#### 컬럼 수정

In [162]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자\n(변경일자),접수번호,접수일자,종류,구분,상세내역,...,Unnamed: 64,자체시험여부,DF 대표,적용기술,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
cert_ld5.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진\n제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '기준치', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 63', 'Unnamed: 64', '자체시험여부', 'DF 대표', '적용기술', 'Unnamed: 68',
 

In [164]:
cert_ld5.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', 
    '최대출력_KW', '최대출력_RPM',
    '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계 형식', '비고', 
    '기준치_CO', '기준치_THC+Nox', '기준치_THC', '기준치_Nox', '기준치_PM', '기준치_PN', '기준치_NH3', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_THC+Nox', '시험결과_ISO_8178_C1_THC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_PN', '시험결과_ISO_8178_C1_NH3', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_연비', '시험결과_ISO_8178_C1_검사일시',
    '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_최대출력_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_THC+Nox', '시험결과_NRSC_THC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_PN', '시험결과_NRSC_NH3', '시험결과_NRSC_CO2', '시험결과_NRSC_연비', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_THC+Nox', '시험결과_NRTC_THC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_PN', '시험결과_NRTC_NH3', '시험결과_NRTC_CO2', '시험결과_NRTC_연비', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm', 
    '자체시험여부', 'DF_대표', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 
    '임의설정자료제출여부'
]

In [165]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 필요한 데이터만 추출(행 제한)

In [166]:
cert_ld5 = cert_ld5.iloc[3:, :]
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
cert_ld5.shape

(2055, 74)

In [167]:
cert_ld5.iloc[:5, :20]

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,엔진형식(엔진모델명),엔진코드,엔진배기량,최대출력_KW,최대출력_RPM,엔진제작사,엔진제작국,건설기계수입사(대표자),건설기계제작사,건설기계명칭
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,F4HGE613I,F4HGE613I*V,6728.00,191,2000,FPT Industrial S.p.A,이태리,해당없음,해당없음,해당없음
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터


In [168]:
cert_ld5.iloc[:5, 20:35]

,건설/농업기계 형식,비고,기준치_CO,기준치_THC+Nox,기준치_THC,기준치_Nox,기준치_PM,기준치_PN,기준치_NH3,시험결과_ISO_8178_C1_CO,시험결과_ISO_8178_C1_THC+Nox,시험결과_ISO_8178_C1_THC,시험결과_ISO_8178_C1_Nox,시험결과_ISO_8178_C1_PM,시험결과_ISO_8178_C1_PN
3,해당없음,NaN,3.50,-,0.19,0.40,0.01,1000000000000,10,-,-,-,-,-,-
4,T7.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Optum 300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T7.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Optum 270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
cert_ld5.iloc[:5, 35:50]

,시험결과_ISO_8178_C1_NH3,시험결과_ISO_8178_C1_CO2,시험결과_ISO_8178_C1_연비,시험결과_ISO_8178_C1_검사일시,시험결과_ISO_8178_C1_최대출력_kw,시험결과_ISO_8178_C1_최대출력_rpm,시험결과_NRSC_CO,시험결과_NRSC_THC+Nox,시험결과_NRSC_THC,시험결과_NRSC_Nox,시험결과_NRSC_PM,시험결과_NRSC_PN,시험결과_NRSC_NH3,시험결과_NRSC_CO2,시험결과_NRSC_연비
3,-,-,-,-,-,-,0.01,-,0.01,0.29,0.00,34600000000,1.52,686.36,212.78
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
cert_ld5.iloc[:5, 50:60]

,시험결과_NRSC_검사일시,시험결과_NRSC_최대출력_kw,시험결과_NRSC_최대출력_rpm,시험결과_NRTC_CO,시험결과_NRTC_THC+Nox,시험결과_NRTC_THC,시험결과_NRTC_Nox,시험결과_NRTC_PM,시험결과_NRTC_PN,시험결과_NRTC_NH3
3,2019-11-14 00:00:00,191,2000,0.02,-,0.01,0.21,0.00,25100000000,0.76
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
cert_ld5.iloc[:5, 60:]

,시험결과_NRTC_CO2,시험결과_NRTC_연비,시험결과_NRTC_검사일시,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,706.07,219.15,2019-11-13 00:00:00,191,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 데이터 확인

In [522]:
set(cert_ld5['비고'].unique())

{nan,
 '건설기계 명칭 정정재교부(22.9.8)',
 '건설기계 수입사 대표자 정정재교부(22.4.29)',
 '건설기계 제작사 대표자 정정재교부(22.11.08)',
 '건설기계 제작사 정정재교부(23.4.4)',
 '건설기계 제작사 정정재교부(23.5.3)',
 '건설기계 제작사명 변경 재교부(23.7.10)',
 '건설기계 형식 정정재교부(21.6.15,교통환경연구소-3376)\n건설기계 수입사 정정재교부(21.8.11, 교통환경연구소-4543)',
 '건설기계 형식 정정재교부(21.9.1,교통환경연구소-4961)',
 '건설기계 형식 정정재교부(22.10.27, 교통환경연구소-6606)',
 '건설기계 형식 정정재교부(22.12.21)',
 '건설기계 형식 정정재교부(22.6.30)',
 '건설기계 형식 정정재교부(22.7.13) / 제작사 정정재교부(23.4.00)',
 '건설기계 형식 정정재교부(22.7.15)',
 '건설기계형식 정정재교부(22.10.7, 교통환경연구소-6230)',
 '건설기계형식 정정재교부(22.3.23, 교통환경연구소-1737)',
 '농업기계형식 정정재교부(22.2.11, 교통환경연구소-1012)',
 '농업기계형식 정정재교부(22.2.11, 교통환경연구소-1012)\n엔진제작사 및 농업기게 제작사 정정재교부(22.10.27,교통환경연구소-6608)',
 '수입사 대표자 변경 재교부(22.10.14)',
 '수입사명 변경 재교부(23.7.10)',
 '엔진제작국 정정재교부(22.10.27, 교통환경연구소-6607)',
 '엔진제작사 정정재교부(23.9.22)',
 '엔진코드 정정재교부(21.3.25)',
 '재생지수 산정 결과 신규인증 결과 대입',
 '정정재교부(교통환경연구소-000,22.5.00)',
 '정정재교부(교통환경연구소-3899,21.7.12)',
 '제작사 정정재교부(23.4.00)',
 '제작사명 정정재교부(23.4.11)',
 '중고',
 '최대출력 정정재교부(22.9.8)',
 '최대출력 정정재교부(

### 데이터 수정(STAGE5)

In [172]:
# 줄 바꿈이 있는 열
# 건설기계수입사(대표자)

# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)

In [173]:
# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)
cert_ld5['건설기계수입사(대표자)'] = cert_ld5['건설기계수입사(대표자)'].str.replace('\n', '')
cert_ld5[['건설기계수입사(대표자)']].head()

,건설기계수입사(대표자)
3,해당없음
4,LS엠트론(구자은)
5,삼정건설기계(유재흥)
6,LS엠트론(구자은)
7,삼정건설기계(유재흥)


#### 컬럼, 데이터 정보들 출력

In [179]:
info01 = cert_ld34.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info01['총 행수'] = cert_ld34.shape[0]
info01['번호'] = [x for x in range(1, info01.shape[0] + 1)]
info01 = info01[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info01

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,0,5971
1,2,상호명,0,5971
2,3,인증번호,0,5971
3,4,최초인증일자,0,5971
4,5,재교부일자(변경일자),0,5971
...,...,...,...,...
59,60,적용기술_DPF,5683,5971
60,61,적용기술_ECU,5692,5971
61,62,적용기술_DOC,5771,5971
62,63,적용기술_기타,5882,5971


In [180]:
info02 = cert_ld5.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info02['총 행수'] = cert_ld5.shape[0]
info02['번호'] = [x for x in range(1, info02.shape[0] + 1)]
info02 = info02[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info02

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,1,2055
1,2,상호명,0,2055
2,3,인증번호,0,2055
3,4,최초인증일자,1,2055
4,5,재교부일자(변경일자),1,2055
...,...,...,...,...
69,70,적용기술_DPF,1925,2055
70,71,적용기술_ECU,1925,2055
71,72,적용기술_DOC,1925,2055
72,73,적용기술_기타,1951,2055


In [182]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [183]:
with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증대장_데이터정보({today_date}).xlsx')) as writer:
    info01.to_excel(writer, sheet_name='TIER-3~TIER-4', index=False)
    info02.to_excel(writer, sheet_name='STAGE5', index=False)

## 전문정비(STD_TB_ERR_RES)

In [558]:
er_dict = {
    'DRVNG_DSTNC':'주행거리', 
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'LOD_COX_MEVLU':'부하일산화탄소측정치', 
    'LOD_HC_MEVLU':'부하탄화수소측정치', 
    'LOD_NOX_MEVLU':'부하질소산화물측정치', 
    'LOD_SMO_MEVLU':'부하매연측정치', 
    'LOD_TQ_MEVLU':'부하토크측정치', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
    'NOLOD_COX_MEVLU':'무부하일산화탄소측정치', 
    'NOLOD_HC_MEVLU':'무부하탄화수소측정치', 
    'NOLOD_AIR_EXCRT_MEVLU':'무부하공기과잉률측정치', 
    'NOLOD_SMO_MEVLU':'무부하매연측정치', 
}

In [559]:
# 0.3s
name = 'STD_TB_ERR_RES(20231011)'
file_name = f'{name}.csv'
er = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DRVNG_DSTNC            54617 non-null  float64
 1   VHRNO                  54617 non-null  object 
 2   VIN                    51051 non-null  object 
 3   LOD_COX_MEVLU          53884 non-null  object 
 4   LOD_HC_MEVLU           53886 non-null  object 
 5   LOD_NOX_MEVLU          53883 non-null  object 
 6   LOD_SMO_MEVLU          53941 non-null  object 
 7   LOD_TQ_MEVLU           53819 non-null  object 
 8   OVHUL_YMD              54617 non-null  int64  
 9   SPCY_OVHUL_BIZPLC_NM   54617 non-null  object 
 10  OVHUL_RCPT_NO          54617 non-null  int64  
 11  PRCINSP_RINSP_RCPT_NO  54617 non-null  object 
 12  NOLOD_COX_MEVLU        53870 non-null  object 
 13  NOLOD_HC_MEVLU         53871 non-null  object 
 14  NOLOD_AIR_EXCRT_MEVLU  28484 non-null  object 
 15  NO

In [560]:
er = er.rename(columns=er_dict)
er.columns

Index(['주행거리', '자동차등록번호', '차대번호', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치',
       '부하매연측정치', '부하토크측정치', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호',
       '무부하일산화탄소측정치', '무부하탄화수소측정치', '무부하공기과잉률측정치', '무부하매연측정치'],
      dtype='object')

### 데이터 확인

In [575]:
set(er['정비접수번호'].unique())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

#### 컬럼, 데이터 정보들 출력

In [562]:
er_cols = pd.DataFrame({'컬럼명':er_dict.keys(), '컬럼내용':er_dict.values()})
er_cols

,컬럼명,컬럼내용
0,DRVNG_DSTNC,주행거리
1,VHRNO,자동차등록번호
2,VIN,차대번호
3,LOD_COX_MEVLU,부하일산화탄소측정치
4,LOD_HC_MEVLU,부하탄화수소측정치
5,LOD_NOX_MEVLU,부하질소산화물측정치
6,LOD_SMO_MEVLU,부하매연측정치
7,LOD_TQ_MEVLU,부하토크측정치
8,OVHUL_YMD,정비일자
9,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명


In [566]:
temp = er.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = er.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(er_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,DRVNG_DSTNC,주행거리,0,54617
1,2,VHRNO,자동차등록번호,0,54617
2,3,VIN,차대번호,3566,54617
3,4,LOD_COX_MEVLU,부하일산화탄소측정치,733,54617
4,5,LOD_HC_MEVLU,부하탄화수소측정치,731,54617
5,6,LOD_NOX_MEVLU,부하질소산화물측정치,734,54617
6,7,LOD_SMO_MEVLU,부하매연측정치,676,54617
7,8,LOD_TQ_MEVLU,부하토크측정치,798,54617
8,9,OVHUL_YMD,정비일자,0,54617
9,10,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,0,54617


In [567]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [568]:
ttemp.to_excel(os.path.join(an4_fold, f'전문정비_데이터정보({today_date}).xlsx'), index=False)

## 전문정비상세(STD_TB_ERR_RES_DET)

In [554]:
erd_dict = {
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'OVHUL_CN':'정비내용', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_CMPNT_NM':'정비부품명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
}

In [555]:
# 0.3s
name = 'STD_TB_ERR_RES_DET(20231011)'
file_name = f'{name}.csv'
erd = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
erd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106819 entries, 0 to 106818
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VHRNO                  106818 non-null  object 
 1   VIN                    94670 non-null   object 
 2   OVHUL_CN               106817 non-null  object 
 3   OVHUL_YMD              106818 non-null  float64
 4   SPCY_OVHUL_BIZPLC_NM   106818 non-null  object 
 5   OVHUL_CMPNT_NM         106817 non-null  object 
 6   OVHUL_RCPT_NO          106818 non-null  float64
 7   PRCINSP_RINSP_RCPT_NO  0 non-null       float64
dtypes: float64(3), object(5)
memory usage: 6.5+ MB


In [556]:
erd = erd.rename(columns=erd_dict)
erd.columns

Index(['자동차등록번호', '차대번호', '정비내용', '정비일자', '전문정비사업소명', '정비부품명', '정비접수번호',
       '정밀재검사접수번호'],
      dtype='object')

### 데이터 확인

In [557]:
erd.shape

(106819, 8)

In [581]:
set(erd['정비접수번호'].unique())

{nan}

#### 컬럼, 데이터 정보들 출력

In [569]:
erd_cols = pd.DataFrame({'컬럼명':erd_dict.keys(), '컬럼내용':erd_dict.values()})
erd_cols

,컬럼명,컬럼내용
0,VHRNO,자동차등록번호
1,VIN,차대번호
2,OVHUL_CN,정비내용
3,OVHUL_YMD,정비일자
4,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명
5,OVHUL_CMPNT_NM,정비부품명
6,OVHUL_RCPT_NO,정비접수번호
7,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호


In [570]:
temp = erd.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = erd.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(erd_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,VHRNO,자동차등록번호,1,106819
1,2,VIN,차대번호,12149,106819
2,3,OVHUL_CN,정비내용,2,106819
3,4,OVHUL_YMD,정비일자,1,106819
4,5,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,1,106819
5,6,OVHUL_CMPNT_NM,정비부품명,2,106819
6,7,OVHUL_RCPT_NO,정비접수번호,1,106819
7,8,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호,106819,106819


In [572]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.31'

In [571]:
ttemp.to_excel(os.path.join(an4_fold, f'전문정비상세_데이터정보({today_date}).xlsx'), index=False)

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)

In [525]:
# 11.7s
name = 'STD_DLM_TB_ERP_GUBUN_BAECHUL(20231011)'
file_name = f'{name}.csv'
gb = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321945 entries, 0 to 1321944
Data columns (total 39 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AIR_PSSR                   1152822 non-null  float64
 1   AIR_TMPRT                  1152822 non-null  float64
 2   CO2_VAL                    1110507 non-null  float64
 3   DRVNG_DSTNC                1320563 non-null  float64
 4   FNCINSP_JGMT_YN            1321919 non-null  object 
 5   INSPOF_NM                  1321945 non-null  object 
 6   INSPOF_RPRSV_NM            1318198 non-null  object 
 7   MAXM_OTPT_MSRMT_VAL        445619 non-null   float64
 8   MAXM_OTPT_PRMT_VAL         544747 non-null   float64
 9   O2_VAL                     1110507 non-null  float64
 10  PRCINSP_YMD                1321945 non-null  int64  
 11  PRCINSP_VLD_YMD            127529 non-null   float64
 12  PRCINSP_VLD_WHL            1321945 non-null  object 
 13  PRCINSP_JGMT

In [ ]:
gb.shape

(1321945, 39)

In [ ]:
gb.columns

Index(['AIR_PSSR', 'AIR_TMPRT', 'CO2_VAL', 'DRVNG_DSTNC', 'FNCINSP_JGMT_YN',
       'INSPOF_NM', 'INSPOF_RPRSV_NM', 'MAXM_OTPT_MSRMT_VAL',
       'MAXM_OTPT_PRMT_VAL', 'O2_VAL', 'PRCINSP_YMD', 'PRCINSP_VLD_YMD',
       'PRCINSP_VLD_WHL', 'PRCINSP_JGMT', 'PRCINSP_KND', 'PRCINSP_RCPT_NO',
       'RINSP_WHL', 'VHCL_PURPS_CD', 'VHRNO', 'VIN',
       'STRCT_CHG_INSP_MSRMT_MTHD', 'NOLOD_SMO_MSRMT_VAL1',
       'NOLOD_SMO_PRMT_VAL1', 'NOLOD_SMO_MSRMT_VAL2', 'NOLOD_SMO_PRMT_VAL2',
       'NOLOD_SMO_MSRMT_VAL3', 'NOLOD_SMO_PRMT_VAL3', 'NOLOD_SMO_MSRMT_VAL4',
       'NOLOD_SMO_PRMT_VAL4', 'NOLOD_SMO_MSRMT_VAL5', 'NOLOD_SMO_PRMT_VAL5',
       'NOLOD_SMO_MSRMT_VAL6', 'NOLOD_SMO_PRMT_VAL6', 'NOLOD_SMO_JGMT_YN1',
       'NOLOD_SMO_JGMT_YN2', 'NOLOD_SMO_JGMT_YN3', 'NOLOD_SMO_JGMT_YN4',
       'NOLOD_SMO_JGMT_YN5', 'NOLOD_SMO_JGMT_YN6'],
      dtype='object')

### 데이터 확인

In [ ]:
gb.iloc[:, :10].head()

,AIR_PSSR,AIR_TMPRT,CO2_VAL,DRVNG_DSTNC,FNCINSP_JGMT_YN,INSPOF_NM,INSPOF_RPRSV_NM,MAXM_OTPT_MSRMT_VAL,MAXM_OTPT_PRMT_VAL,O2_VAL
0,0.00,0.00,14.80,25377.00,Y,이현자동차검사소,최원수,NaN,NaN,0.00
1,101.30,18.00,0.00,172542.00,Y,서인천자동차검사소,박상영,NaN,NaN,0.00
2,98.20,24.80,0.00,203176.00,Y,용인자동차검사소,최광진,NaN,NaN,0.00
3,101.20,15.20,0.00,208787.00,Y,노원자동차검사소,김동연,NaN,NaN,0.00
4,99.80,13.40,0.00,158139.00,Y,성동자동차검사소,나성철,NaN,NaN,0.00


In [ ]:
set(gb.FNCINSP_JGMT_YN.unique())

{' ', '8', 'N', 'Y', nan}

In [ ]:
gb.iloc[:, 10:20].head()

,PRCINSP_YMD,PRCINSP_VLD_YMD,PRCINSP_VLD_WHL,PRCINSP_JGMT,PRCINSP_KND,PRCINSP_RCPT_NO,RINSP_WHL,VHCL_PURPS_CD,VHRNO,VIN
0,20121031,NaN,~,Y,A21C02,27302012103101971,20121031~20121109,A20P,43버6629,KMHFG41EBCA120332
1,20121031,NaN,~,Y,A21C02,28402012103101621,20121031~20121109,A20P,81노1182,KMFLA18EP6C009216
2,20121031,NaN,~,Y,A21C02,41j02012103101191,20121031~20121109,A20P,경기94다4315,KN3HDP3352K836554
3,20121031,NaN,~,Y,A21C02,11602012103102571,20121031~20121109,A20P,서울34머6188,KMHWP81HP2U498130
4,20121031,NaN,~,Y,A21C02,11a02012103102091,20121031~20121109,A20P,80조2956,KMFWVH7HP5U646771


In [ ]:
set(gb.VHCL_PURPS_CD.unique())

{'A20B', 'A20O', 'A20P', 'A20T'}

In [ ]:
gb.iloc[:, 20:30].head()

,STRCT_CHG_INSP_MSRMT_MTHD,NOLOD_SMO_MSRMT_VAL1,NOLOD_SMO_PRMT_VAL1,NOLOD_SMO_MSRMT_VAL2,NOLOD_SMO_PRMT_VAL2,NOLOD_SMO_MSRMT_VAL3,NOLOD_SMO_PRMT_VAL3,NOLOD_SMO_MSRMT_VAL4,NOLOD_SMO_PRMT_VAL4,NOLOD_SMO_MSRMT_VAL5
0,부하검사(ASM-Idling),0.00,0.60,7.00,120.00,0.00,790.00,0.00,1.00,8.00
1,부하검사(LUG DOWN),0.00,20.00,0.00,20.00,0.00,20.00,2504.00,5.00,123.80
2,부하검사(KD-147),14.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,부하검사(KD-147),1.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,부하검사(KD-147),0.00,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
set(gb.STRCT_CHG_INSP_MSRMT_MTHD.unique())

{'무부하검사(TSI)',
 '무부하검사(급가속)',
 '무부하검사(정지가동)',
 '부하검사(ASM-Idling)',
 '부하검사(ASM2525)',
 '부하검사(KD-147)',
 '부하검사(LUG DOWN)'}

In [ ]:
gb.iloc[:, 30:40].head()

,NOLOD_SMO_PRMT_VAL5,NOLOD_SMO_MSRMT_VAL6,NOLOD_SMO_PRMT_VAL6,NOLOD_SMO_JGMT_YN1,NOLOD_SMO_JGMT_YN2,NOLOD_SMO_JGMT_YN3,NOLOD_SMO_JGMT_YN4,NOLOD_SMO_JGMT_YN5,NOLOD_SMO_JGMT_YN6
0,120.00,0.98,0.10,Y,Y,Y,Y,Y,Y
1,117.00,NaN,NaN,Y,Y,Y,Y,Y,NaN
2,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN


In [ ]:
set(gb.NOLOD_SMO_JGMT_YN6.unique())

{' ', '0', 'N', 'Y', nan}

#### 컬럼, 데이터 정보들 출력

In [ ]:
temp = gb.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = gb.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
temp = temp[['번호', '컬럼명', '빈 값 개수', '총 행수']]
temp

,번호,컬럼명,빈 값 개수,총 행수
0,1,AIR_PSSR,169123,1321945
1,2,AIR_TMPRT,169123,1321945
2,3,CO2_VAL,211438,1321945
3,4,DRVNG_DSTNC,1382,1321945
4,5,FNCINSP_JGMT_YN,26,1321945
5,6,INSPOF_NM,0,1321945
6,7,INSPOF_RPRSV_NM,3747,1321945
7,8,MAXM_OTPT_MSRMT_VAL,876326,1321945
8,9,MAXM_OTPT_PRMT_VAL,777198,1321945
9,10,O2_VAL,211438,1321945


In [ ]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

In [ ]:
temp.to_excel(os.path.join(an5_fold, f'구조변경_데이터정보({today_date}).xlsx'), index=False)

# EDA

## 전문정비(STD_TB_ERR_RES)

In [17]:
er['정비일자'].isnull().sum()

0

In [15]:
er['정비일자'].describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 정비일자, dtype: float64

In [19]:
er['정비일자'] = pd.to_datetime(er['정비일자'], format='%Y%m%d', errors='coerce')
er['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-07
Name: 정비일자, dtype: datetime64[ns]

In [20]:
er['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\328505498.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  er['정비일자'].describe()


count                   54617
unique                   1839
top       2007-10-02 00:00:00
freq                      123
first     2006-07-27 00:00:00
last      2012-10-10 00:00:00
Name: 정비일자, dtype: object

## 전문정비상세(STD_TB_ERR_RES_DET)

In [18]:
erd['정비일자'].isnull().sum()

1

In [16]:
erd['정비일자'].describe()

count     106818.00
mean    20083911.85
std        15852.89
min     20060727.00
25%     20070829.00
50%     20080529.00
75%     20090921.00
max     20120928.00
Name: 정비일자, dtype: float64

In [21]:
erd['정비일자'] = pd.to_datetime(erd['정비일자'], format='%Y%m%d', errors='coerce')
erd['정비일자'].head()

0   2007-02-08
1   2006-08-11
2   2006-08-11
3   2006-08-11
4   2006-12-11
Name: 정비일자, dtype: datetime64[ns]

In [22]:
erd['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\338169835.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  erd['정비일자'].describe()


count                  106818
unique                   1832
top       2007-08-13 00:00:00
freq                      232
first     2006-07-27 00:00:00
last      2012-09-28 00:00:00
Name: 정비일자, dtype: object

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)